In [ ]:
import pandas as pd
#from pandas.core.common import SettingWithCopyWarning
import warnings
from pandas.errors import SettingWithCopyWarning
import math
import statistics
import warnings
from difflib import SequenceMatcher

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Loading data files

### Licensed vehicles

In [ ]:
# Licensing data
licensed_vehicle_df = pd.concat([pd.read_csv(am__licensed_path),
                               pd.read_csv(nz__licensed_path)])

In [ ]:
print(len(licensed_vehicle_df))

In [ ]:
licensed_vehicle_df.head()

## Producing a dictionary with all of the makes, models and years.

In [ ]:
make_model_year = {}
for i, row in licensed_vehicle_df.iterrows():
    if i % 20000 == 0:
        print(i)
    make = row["Make"]
    model = row["GenModel"]
    year = row["YearManufacture"]
    if make not in make_model_year.keys():
        make_model_year[make] = {}
    if model not in make_model_year[make]:
        make_model_year[make][model] = []
    if year not in make_model_year[make][model]:
        make_model_year[make][model].append(year)

In [ ]:
make_model_year

## Getting counts of all Makes in each year separately

In [ ]:
# Full make count
full_make_count = {2022: {}, 2021: {}, 2020: {}, 2019: {}, 2018: {}}
for i, row in licensed_vehicle_df.iterrows():
    if i % 20000 == 0:
        print(i)
    make = row["Make"]
    for k in full_make_count:
        if make not in full_make_count[k].keys():
            full_make_count[k][make] = 0
        if row[str(k)] != "[z]":
            full_make_count[k][make] += int(row[str(k)])

In [ ]:
full_make_count

## Getting counts of all Models in each year separately

In [ ]:
# Full model count
full_model_count = {2022: {}, 2021: {}, 2020: {}, 2019: {}, 2018: {}}
for i, row in licensed_vehicle_df.iterrows():
    if i % 20000 == 0:
        print(i)
    model = row["GenModel"]
    for k in full_model_count:
        if model not in full_model_count[k].keys():
            full_model_count[k][model] = 0
        if row[str(k)] != "[z]":
            full_model_count[k][model] += int(row[str(k)])

In [ ]:
full_model_count

In [ ]:
licensed_vehicle_df

## Count of each Make that are licensed on the road each year.

In [ ]:
# Licensed make count
licensed_vehicle_df[licensed_vehicle_df["LicenceStatus"]=="Licensed"]
make_count = {2022: {}, 2021: {}, 2020: {}, 2019: {}, 2018: {}}
for i, row in licensed_vehicle_df.iterrows():
    if i % 20000 == 0:
        print(i)
    make = row["Make"]
    for k in make_count:
        if make not in make_count[k].keys():
            make_count[k][make] = 0
        if row[str(k)] != "[z]":
            make_count[k][make] += int(row[str(k)])

## Count of each Model that are licensed on the road each year.

In [ ]:
# Licensed model count
model_count = {2022: {}, 2021: {}, 2020: {}, 2019: {}, 2018: {}}
for i, row in licensed_vehicle_df.iterrows():
    if i % 20000 == 0:
        print(i)
    model = row["GenModel"]
    for k in model_count:
        if model not in model_count[k].keys():
            model_count[k][model] = 0
        if row[str(k)] != "[z]":
            model_count[k][model] += int(row[str(k)])

In [ ]:
model_count

### Vehicle incident data

    - Splitting data into two years 2020 and 2021

In [ ]:
# Accident-Vehicle data
vehicle_accident_df = pd.read_csv(accident_vehicles_path)
vehicle_accident_original_df = vehicle_accident_df
print(len(vehicle_accident_df))
vehicle_accident_df = {2020: vehicle_accident_df[vehicle_accident_df["accident_year"]==2020], 2021:vehicle_accident_df[vehicle_accident_df["accident_year"]==2021]}
print(len(vehicle_accident_df[2020])+len(vehicle_accident_df[2021]))

In [ ]:
vehicle_accident_df

In [ ]:
vehicle_accident_original_df.head()

## Joining the accident reference with the vehicle make model description

In [ ]:
vehicle_accident_refs = {2020: {}, 2021: {}}
years = [2020, 2021]
for y in years:
    for i, row in vehicle_accident_df[y].iterrows():
        veh = row["generic_make_model"]
        if i % 20000 == 0:
            print(i)
        if veh not in vehicle_accident_refs[y].keys():
            vehicle_accident_refs[y][veh] = []
        vehicle_accident_refs[y][veh].append(row["accident_reference"])
print(len(vehicle_accident_refs[2020]))
print(len(vehicle_accident_refs[2021]))

In [ ]:
vehicle_accident_refs

## Finding which vehicles from the licensed vehicles dataset appears in the vehicle incident data.

In [ ]:
matches = list(set(licensed_vehicle_df["GenModel"]).intersection(set(list(vehicle_accident_df[2020]["generic_make_model"])+list(vehicle_accident_df[2021]["generic_make_model"]))))
print(len(matches))
print(len(set(licensed_vehicle_df["Make"])))
len(set(licensed_vehicle_df[licensed_vehicle_df["GenModel"].isin(matches)]["Make"]))

In [ ]:
matches

#### Accident data

In [ ]:
# Accident data
accident_df = pd.read_csv(accident_file_path)
accident_factors = ["accident_severity", "number_of_vehicles", "number_of_casualties", "speed_limit", "road_type", "speed_limit",
                    "light_conditions", "weather_conditions", "road_surface_conditions", "special_conditions_at_site", "carriageway_hazards"]
speed_orders = list(accident_df["speed_limit"].value_counts().keys())

### accident_data.csv **vs** accident_vehicles.csv

In [ ]:
accident_df.head()

In [ ]:
len(accident_df)

In [ ]:
vehicle_accident_original_df.head()

In [ ]:
len(vehicle_accident_original_df)

## Calculating an accident value for each accident reference

    - Main calculation occurs here: where the columns 'accident_severity', 'number_of_vehicles', 'number_of_casualties', 'road_surface_conditions', 'weather_conditions', 'light_conditions' and 'speed_limit' are all combined to produce an overall rating for each of the accidents in the data (192,274).

In [ ]:
accident_values = {}
for i, row in accident_df.iterrows():
    if i % 10000 == 0:
        print(i)
    ref = row["accident_reference"]
    value = 33 if row["accident_severity"] == 3 else 16 if row["accident_severity"] == 2 else 0
    # alter values to add to 17
    value += ((5-row["number_of_vehicles"])*2)/2 if row["number_of_vehicles"] <= 4 else 0
    if row["number_of_vehicles"] <= 4:
        value += (row["number_of_casualties"])/2 if row["number_of_casualties"] <= 10 else 10/2
    value += 4/2 if row["road_surface_conditions"] == 1 else 3/2 if row["road_surface_conditions"] == 2 or row["road_surface_conditions"] == 7 else 2/2 if row["road_surface_conditions"] == 3 else 1/2 if row["road_surface_conditions"] < 6 else 0 
    value += 3/2 if row["weather_conditions"] == 1 else 2/2 if row["weather_conditions"] < 4 else 1/2 if row["weather_conditions"] < 6 else 0
    value += 3/2 if row["light_conditions"] == 1 else 2/2 if row["light_conditions"] == 4 else 1/2 if row["light_conditions"] == 5 else 0
    value += int((speed_orders.index(row["speed_limit"]))/2)/2 if row["speed_limit"] > 0 else 0
    accident_values[ref] =  value

In [ ]:
accident_values

In [ ]:
len(accident_values)

In [ ]:
vehicle_accident_refs

- Calculating the commonality of each make and model in the accident data

In [ ]:
vehicle_accident_value_list = {2020: {}, 2021: {}}
commonality = {2020: {}, 2021: {}}
for y in years:
    for k1 in vehicle_accident_refs[y]:
        values = []
        for k2 in vehicle_accident_refs[y][k1]:
            if str(k2) in accident_values:
                values.append(accident_values[str(k2)])
        if len(values) > 0 and k1 in model_count[y].keys() and model_count[y][k1] > 0:
            commonality[y][k1] = (len(values)/model_count[y][k1])
            vehicle_accident_value_list[y][k1] = values

In [ ]:
# 50 total for rarity factor
mx = [sum([x for x in commonality[2020].values() if x < 1])/len([x for x in commonality[2020].values() if x < 1]),
      sum([x for x in commonality[2021].values() if x < 1])/len([x for x in commonality[2021].values() if x < 1])] 
for i in range(len(years)):
    for k in commonality[years[i]].keys():
        commonality[years[i]][k] = (commonality[years[i]][k]/mx[i])*50
        commonality[years[i]][k] = commonality[years[i]][k] if commonality[years[i]][k] < 50 else 50

In [ ]:
vehicle_accident_values = {}
for y in years:
    for k in vehicle_accident_value_list[y]:
        if k not in vehicle_accident_values.keys():
            vehicle_accident_values[k] = []
        for val in vehicle_accident_value_list[y][k]:
            vehicle_accident_values[k].append(int(val+commonality[y][k]))
for k in vehicle_accident_values:
    length = (int(len(vehicle_accident_values[k])/5)*3)
    sort = sorted(vehicle_accident_values[k])[0:length if length > 0 else 1]
    vehicle_accident_values[k] = int(sum(sort)/len(sort))

In [ ]:
accident_values_df = pd.DataFrame(columns=["GenModel","Accident_rating"])
accident_values_df = accident_values_df.assign(GenModel=vehicle_accident_values.keys(), Accident_rating=vehicle_accident_values.values())

In [ ]:
accident_values_df

#### Casualty data

In [ ]:
# Casualty data
casualty_df = pd.read_csv(casualty_data_path)
unique_casualties = set(casualty_df["accident_reference"])

In [ ]:
casualties = {}
for i, row in casualty_df.iterrows():
    if i % 20000 == 0:
        print(i)
    ref = row["accident_reference"]
    if ref not in casualties.keys():
        casualties[ref] = []
    casualties[ref].append(row)
print(len(casualties))

- This is the main chunk in assigning a 'casualty value' for each accident in the injury data.

In [ ]:
casualty_values = {}
for ref in casualties.keys():
    if i % 10000 == 0:
        print(i)
    value = (len(casualties[ref])-1)*3 if len(casualties[ref]) <= 5 else 16
    severities = []
    for row in casualties[ref]:
        severities.append(row["casualty_severity"])
    value += max(severities)*4
    severities = sorted(severities)[-3:-1]
    for v in severities:
        value += v
    classes = []
    for row in casualties[ref]:
        classes.append(row["casualty_class"])
    if 1 in classes:
        value += 4
    passenger_count = classes.count(2)
    value += passenger_count*2 if passenger_count < 3 else 4
    pedestrian_count = classes.count(3)
    value += pedestrian_count*2 if pedestrian_count < 5 else 8
    casualty_values[ref] =  [value, len(casualties[ref])]

In [ ]:
vehicle_casualty_value_list = {2020: {}, 2021: {}}
commonality = {2020: {}, 2021: {}}
for y in years:
    for k1 in vehicle_accident_refs[y]:
        values = []
        for k2 in vehicle_accident_refs[y][k1]:
            if str(k2) in casualty_values:
                values.append(casualty_values[str(k2)])
        if len(values) > 0 and k1 in model_count[y].keys() and model_count[y][k1] > 0:
            commonality[y][k1] = (len(values)/model_count[y][k1])
            vehicle_casualty_value_list[y][k1] = values

In [ ]:
# 30 total for rarity factor
mx = [sum([x for x in commonality[2020].values() if x < 1])/len([x for x in commonality[2020].values() if x < 1]),
      sum([x for x in commonality[2021].values() if x < 1])/len([x for x in commonality[2021].values() if x < 1])] 
for i in range(len(years)):
    for k in commonality[years[i]].keys():
        commonality[years[i]][k] = (commonality[years[i]][k]/mx[i])*30
        commonality[years[i]][k] = int(commonality[years[i]][k]) if commonality[years[i]][k] < 30 else 30
        involvement_count = []
        if k in vehicle_casualty_value_list[years[i]]:
            for v in vehicle_casualty_value_list[years[i]][k]:
                involvement_count.append(v[1])
        else:
            involvement_count.append(0)
        involvement_count = statistics.mean(involvement_count)*20
        involvement_count = involvement_count if involvement_count < 20 else 20
        commonality[years[i]][k] = commonality[years[i]][k]+involvement_count

In [ ]:
commonality

In [ ]:
vehicle_casualty_values = {}
for y in years:
    for k in vehicle_casualty_value_list[y]:
        if k not in vehicle_casualty_values.keys():
            vehicle_casualty_values[k] = []
        for val in vehicle_casualty_value_list[y][k]:
            vehicle_casualty_values[k].append(int(val[0]+commonality[y][k]))
for k in vehicle_casualty_values:
    length = (int(len(vehicle_casualty_values[k])/5)*3)
    sort = sorted(vehicle_casualty_values[k])[(-1*length if length > 0 else 0):]
    vehicle_casualty_values[k] = int(sum(sort)/len(sort))

In [ ]:
set(vehicle_casualty_values.values())

In [ ]:
casualty_values_df = pd.DataFrame(columns=["GenModel","Casualty_rating"])
casualty_values_df = casualty_values_df.assign(GenModel=vehicle_casualty_values.keys(), Casualty_rating=vehicle_casualty_values.values())

#### Theft data

    - Not given information about whether the vehicle is licensed.

In [ ]:
# Theft data
theft_original_df = pd.read_csv(theft_file_path)

theft_df = theft_original_df.copy()

In [ ]:
theft_df

In [ ]:
theft_df = theft_df.fillna(0)
theft_df

## Dealing with NaN's

- A blank value represents 0/no thefts

In [ ]:
theft_original_df

In [ ]:
theft_original_df.loc[theft_original_df['Make/Model']=='ABARTH 595 TROFEO']

In [ ]:
for index, row in theft_original_df.iterrows():
    val_2022 = row['2022']
    val_2021 = row['2021']
    val_2020 = row['2020']
    val_2019 = row['2019']
    val_2018 = row['2018']
    
    make_model = row['Make/Model']
    
    if pd.isnull(val_2022)==False and pd.isnull(val_2021)==True and pd.isnull(val_2020)==False:
        print(make_model)
    

## Time trends

In [ ]:
years = [2018, 2019, 2020, 2021, 2022]

total_thefts = []

for val in years:
    total_thefts.append(theft_original_df[str(val)].sum())
total_thefts

    

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
plt.plot(years, total_thefts)
plt.xticks([2018, 2019, 2020, 2021, 2022]);

In [ ]:
for val in theft_years_summed_df['Licensed_Manfacturer'].unique():
    for val2 in upper_manufacturers:
        if val != val2 and fuzz.token_set_ratio(val, val2)>80:
            print(val)
            print(val2)
            

In [ ]:
theft_years_summed_with_make_and_model_df

In [ ]:
upper_manufacturers = [val.upper() for val in current_vehicle_df['Manufacturer'].unique()]

upper_manufacturers_consider = upper_manufacturers + ['ROLLS ROYCE', 'MG', 'MERCEDES', 'LAND ROVER', 'DOUGLAS']

In [ ]:
len(upper_manufacturers_consider)

In [ ]:
vehicle_file_relevant_df = theft_years_summed_df.loc[theft_years_summed_df['Licensed_Manfacturer']!='UNKNOWN']


vehicle_file_relevant_df = vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Licensed_Manfacturer'].isin(upper_manufacturers_consider)]

vehicle_file_relevant_df

In [ ]:
vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Licensed_GenModel']=='YAMAHA T'].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Licensed_GenModel']=='YAMAHA MT'].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Licensed_GenModel']=='ZONTES ZT'].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Licensed_GenModel']=='WLIE VERACRUZ'].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Licensed_GenModel']=='WK BIKES RT 125'].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Make/Model'].str.contains('HARLEY-DAVIDSON')].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Make/Model'].str.contains('KAWASAKI')].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Make/Model'].str.contains('SACHS')].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Make/Model'].str.contains('SANBEN')].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Make/Model'].str.contains('VESPA')].index)

vehicle_file_relevant_df = vehicle_file_relevant_df.drop(vehicle_file_relevant_df.loc[vehicle_file_relevant_df['Make/Model'].str.contains('WACKER')].index)


In [ ]:
vehicle_file_relevant_df['Licensed_Manfacturer'].unique()

In [ ]:
grouped_by_GenModel = vehicle_file_relevant_df.groupby(by=['Licensed_Manfacturer', 'Licensed_GenModel']).agg({'2022':'sum', '2021':'sum', '2020':'sum', '2019':'sum', '2018':'sum'}).reset_index()

grouped_by_GenModel

## Picking out values where all years have data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
grouped_by_GenModel_no_zeros

- AC is messed up

In [ ]:
upper_manufacturers_consider.sort(key=len, reverse=True)

for index, row in grouped_by_GenModel_no_zeros.loc[grouped_by_GenModel_no_zeros['Licensed_Manfacturer']=='AC'].iterrows():
    matched = 0
    genModel = row['Licensed_GenModel']
    for val in upper_manufacturers_consider:
        if val in genModel:
            grouped_by_GenModel_no_zeros.loc[index, 'Licensed_Manfacturer'] = val
            matched+=1
            break
    
    if matched == 0:
        grouped_by_GenModel_no_zeros = grouped_by_GenModel_no_zeros.drop(index)

In [ ]:
grouped_by_GenModel_no_zeros = grouped_by_GenModel.loc[(grouped_by_GenModel['2022']!=0)&(grouped_by_GenModel['2021']!=0)&(grouped_by_GenModel['2020']!=0)&(grouped_by_GenModel['2019']!=0)&(grouped_by_GenModel['2018']!=0)]
#grouped_by_GenModel_no_zeros = grouped_by_GenModel_no_zeros.set_index('Licensed_GenModel')

grouped_manufacturer = grouped_by_GenModel_no_zeros['Licensed_Manfacturer'].unique()

for manufacturer in grouped_manufacturer[3:5]:
    manufacturer_subset = grouped_by_GenModel_no_zeros.loc[grouped_by_GenModel_no_zeros['Licensed_Manfacturer']==manufacturer]
    all_genmodels = []
    #manufacturer_subset = manufacturer_subset.set_index('Licensed_GenModel')
    plt.figure(figsize=(16,12))
    for index, row in manufacturer_subset.iterrows():
        val_2022 = row['2022']
        val_2021 = row['2021']
        val_2020 = row['2020']
        val_2019 = row['2019']
        val_2019 = row['2018']
        
        GenModel = row['Licensed_GenModel']
        all_genmodels.append(GenModel)
    
        plt.plot([2018, 2019, 2020, 2021, 2022], [val_2018, val_2019, val_2020, val_2021, val_2022])
        
    plt.ylabel('Number of Thefts', fontsize=12)
    plt.xlabel('Years', fontsize=12)
    plt.title('Plot showing the number of thefts for different {} GenModels'.format(manufacturer), fontsize=22)
    plt.legend(all_genmodels)
    plt.show()

## Attempting time series forecasting/smoothing through lockdown

- Use all GenModels-even with zero values

In [ ]:
import pmdarima as pm

from pmdarima.model_selection import train_test_split

In [ ]:
grouped_by_GenModel

In [ ]:
for index, row in grouped_by_GenModel[:6].iterrows():
    
    val_2022 = row['2022']
    #val_2021 = row['2021']
    #val_2020 = row['2020']
    val_2019 = row['2019']
    val_2018 = row['2018']
    
    gen_model = row['Licensed_GenModel']

    thefts_list = [val_2018, val_2019, val_2022]
    
    train, test = train_test_split(thefts_list, train_size=2, test_size=1)
    print(train)
    print(test)
    
    model = pm.auto_arima(train, seasonal=False, m=1, with_intercept=True, n_fits=5, trace=False)
    
    forecasts = model.predict(len(test))
    print(forecasts[0])
    print(test)
    
    forecasts_plus = [train[-1]] + list(forecasts)
    tests_plus = [train[-1]] + list(test)
    
    grouped_by_GenModel.loc[index, 'Theft_Prediction'] = forecasts[0]
    
    # Visualize the forecasts (blue=train, green=forecasts)
    x = np.arange(len(thefts_list))
    plt.plot(range(2018,2022,1), train, c='blue')
    plt.plot(range(2021, 2023, 1), forecasts_plus, c='green')
    plt.plot(range(2021, 2023, 1), tests_plus, c='red')
    plt.xticks(range(2018,2023,1))
    plt.show()

In [ ]:
grouped_by_GenModel

In [ ]:
len(y)

In [ ]:
len(vehicle_file_relevant_df['Licensed_Manfacturer'].unique())

In [ ]:
theft_original_df.loc[theft_original_df['2020']==0]

In [ ]:
theft_df.loc[(theft_df['2022']!=0)|(theft_df['2021']!=0)|(theft_df['2020']!=0)|(theft_df['2019']!=0)|(theft_df['2018']!=0)]

In [ ]:
theft_years_occurences_summed_df = theft_df.groupby(by=['Make/Model', 'Year Manufacture/First Reg']).agg(sum).reset_index()

#Check groupby has worked well

theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Make/Model']=='AUDI TT']

In [ ]:
theft_years_occurences_summed_df

In [ ]:
len(theft_years_occurences_summed_df)

In [ ]:
gen_models_list = list(full_model_count[2022].keys())+list(full_model_count[2021].keys())+list(full_model_count[2020].keys())+list(full_model_count[2019].keys())+list(full_model_count[2018].keys())

gen_models_unique_list = set(gen_models_list)

In [ ]:
print(len(gen_models_list))
print(len(gen_models_unique_list))

# Josh map

In [ ]:
import pickle

In [ ]:
pkl_file = open(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\stolen_vehicle_map.pkl", 'rb')
matched_vehicles = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
matched_vehicles

In [ ]:
theft_years_occurences_summed_df

In [ ]:
ABI_desc_with_manufacturer = []

for key in matched_vehicles.keys():
    values = matched_vehicles[key]
    
    ABI_desc_with_manufacturer.append(values)

ABI_desc_with_manufacturer_flattened = sum(ABI_desc_with_manufacturer, [])

In [ ]:
current_vehicle_df['Veh_Desc_ABI_Manufacturer'] = current_vehicle_df['Manufacturer'] + " " + current_vehicle_df['Veh_Desc_ABI']

current_vehicle_df['Veh_Desc_ABI_Manufacturer'] = current_vehicle_df['Veh_Desc_ABI_Manufacturer'].apply(lambda x: x.upper())


In [ ]:
found_tvi = list(current_vehicle_df.loc[current_vehicle_df['Veh_Desc_ABI_Manufacturer'].isin(ABI_desc_with_manufacturer_flattened)]['TVI_Code'])

missed_tvi = []

for val in current_vehicle_df['TVI_Code']:
    if val not in found_tvi:
        missed_tvi.append(val)

In [ ]:
missed_vehicle_file_df = current_vehicle_df.loc[current_vehicle_df['TVI_Code'].isin(missed_tvi)]
missed_vehicle_file_df.to_csv(missing_vehicles_file_path, index = False)

In [ ]:
missed_vehicle_file_df['Manufacturer'].unique()

In [ ]:
missed_vehicle_file_df.loc[missed_vehicle_file_df['Manufacturer']=='BMW']

In [ ]:
theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Make/Model'].str.contains('BMW X5 I')]

In [ ]:
theft_years_summed_with_make_and_model_df

In [ ]:
theft_years_summed_with_make_and_model_df.loc[theft_years_summed_with_make_and_model_df['Model_Possible']=='CONTINENTAL GT3-R AUTO']

In [ ]:
current_vehicle_df.loc[current_vehicle_df['Veh_Desc_ABI'] == '607 SE HDI']

## Improve formatting

In [ ]:
current_vehicle_file_df = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\08_Master_File\vehicle_features_Vehicle_File_Master_05_09_2023.csv")

In [ ]:
vehicle_features_manufacturers_upper_list = [x.upper() for x in current_vehicle_file_df['Manufacturer'].unique()]


In [ ]:
import numpy as np

from fuzzywuzzy import fuzz

In [ ]:
theft_years_occurences_summed_df['Exact_Match'] = np.zeros(len(theft_years_occurences_summed_df))
theft_years_occurences_summed_df['Possible_Match'] = ''
theft_years_occurences_summed_df['Match_Score'] = np.zeros(len(theft_years_occurences_summed_df))
theft_years_occurences_summed_df['Manufacturer'] = ''
theft_years_occurences_summed_df['Model'] = ''
theft_years_occurences_summed_df['Make/Model'] = theft_years_occurences_summed_df['Make/Model'].apply(lambda x: x.strip())

In [ ]:
licensed_manufacturers = licensed_vehicle_df['Make'].unique()
licensed_models_for_manufacturer = {}
for manufacturer in licensed_manufacturers:
    subset_df = licensed_vehicle_df.loc[licensed_vehicle_df['Make']==manufacturer]
    model_col = subset_df['GenModel'].apply(lambda x: x.replace(manufacturer, '').strip())
    model_list = list(model_col.unique())
    licensed_models_for_manufacturer[manufacturer] = model_list
licensed_models_for_manufacturer


## Lists of manufacturers

In [ ]:
licensed_manufacturers_consider = []

for val in licensed_manufacturers:
    if val in vehicle_features_manufacturers_upper_list:
        licensed_manufacturers_consider.append(val)
print(len(licensed_manufacturers_consider))
print(len(vehicle_features_manufacturers_upper_list))

missed_manufacturers = []

for val in vehicle_features_manufacturers_upper_list:
    if val not in licensed_manufacturers_consider:
        missed_manufacturers.append(val)

list(licensed_manufacturers).sort(key=len, reverse=True)

## Extract Manufacturer

In [ ]:
theft_years_occurences_summed_df['Make/Model'] = theft_years_occurences_summed_df['Make/Model'].apply(lambda x: x.upper())

In [ ]:
def extract_make_func(make_model):
    for val in licensed_manufacturers:
        if val in make_model:
            manufacturer = val
            break
        else:
            manufacturer = 'UNKNOWN'
    return manufacturer

theft_years_occurences_summed_df['Manufacturer'] = theft_years_occurences_summed_df['Make/Model'].apply(lambda x: extract_make_func(x))

In [ ]:
theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Manufacturer']=='UNKNOWN']['Make/Model'].unique()

In [ ]:
theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Manufacturer']!='UNKNOWN']

## Extracting Model

In [ ]:
def extract_model_func(make, make_model):
    if make!='UNKNOWN':
        models = licensed_models_for_manufacturer[make]
        models.sort(key=len, reverse = True)
        for model_val in models:
            if model_val in make_model:
                model = model_val
                break
            else:
                model = 'UNKNOWN'
        
    else:
        model = 'UNKNOWN'
    return model

theft_years_occurences_summed_df['Model'] = theft_years_occurences_summed_df.apply(lambda x: extract_model_func(x['Manufacturer'], x['Make/Model']), axis=1)

In [ ]:
theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Model']!='UNKNOWN']


In [ ]:
licensed_models_for_manufacturer['BMW']

In [ ]:
models = list(set(licensed_vehicle_df["GenModel"]))
matched = {}
x = 0
for i in range(len(models)):
    if i % 100 == 0:
        print(i)
    matched[models[i]] = list(theft_years_occurences_summed_df[theft_years_occurences_summed_df["Make/Model"].apply(lambda x: models[i].upper() in x.upper())]["Make/Model"])


In [ ]:
theft_years_occurences_summed_df['Make/Model'] = theft_years_occurences_summed_df['Make/Model'].apply(lambda x: x.upper())

models = list(set(licensed_vehicle_df["GenModel"]))

models_upper = [x.upper() for x in models]

models_upper.sort(key=len, reverse=True)

def search_for_GenModel(make_model):
    for model in models:
        if model in make_model:
            licensed_make_model = model
            break
        else:
            licensed_make_model = 'UNKNOWN'
    return licensed_make_model

theft_years_occurences_summed_df['Licensed_GenModel'] = theft_years_occurences_summed_df['Make/Model'].apply(lambda x: search_for_GenModel(x))
    

In [ ]:
theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Licensed_GenModel']!='UNKNOWN']

- Applying Manufacturers and Models found separately to increase number of GenModels found

In [ ]:
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']!='UNKNOWN')&(theft_years_occurences_summed_df['Model']!='UNKNOWN'), 'Licensed_GenModel'] = theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']!='UNKNOWN')&(theft_years_occurences_summed_df['Model']!='UNKNOWN')]['Manufacturer'] + theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']!='UNKNOWN')&(theft_years_occurences_summed_df['Model']!='UNKNOWN')]['Model']


In [ ]:
#Fixing Manufacturer
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('AUDI')), 'Manufacturer'] = 'AUDI'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('BMW')), 'Manufacturer'] = 'BMW'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('CADILLAC')), 'Manufacturer'] = 'CADILLAC'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('JAGUAR')), 'Manufacturer'] = 'JAGUAR'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('VOLKSWAGEN')), 'Manufacturer'] = 'VOLKSWAGEN'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('VAUXHALL')), 'Manufacturer'] = 'VAUXHALL'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('TOYOTA')), 'Manufacturer'] = 'TOYOTA'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('ROVER')), 'Manufacturer'] = 'ROVER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('RENAULT')), 'Manufacturer'] = 'RENAULT'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('NISSAN')), 'Manufacturer'] = 'NISSAN'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('MERCEDES-BENZ')), 'Manufacturer'] = 'MERCEDES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('LAND ROVER')), 'Manufacturer'] = 'LAND ROVER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('LEXUS')), 'Manufacturer'] = 'LEXUS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('KIA')), 'Manufacturer'] = 'KIA'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('FIAT')), 'Manufacturer'] = 'FIAT'

theft_years_occurences_summed_df['Model_Possible'] = theft_years_occurences_summed_df.apply(lambda x: x['Make/Model'].replace(x['Manufacturer'], '').strip(), axis = 1)

#Fixing Licensed_GenModel

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ABARTH')&(theft_years_occurences_summed_df['Make/Model'].str.contains('GRANDE PUNTO')), 'Licensed_GenModel'] = 'ABARTH PUNTO'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('1')), 'Licensed_GenModel'] = 'BMW 1 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('2')), 'Licensed_GenModel'] = 'BMW 2 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('3')), 'Licensed_GenModel'] = 'BMW 3 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('4')), 'Licensed_GenModel'] = 'BMW 4 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('5')), 'Licensed_GenModel'] = 'BMW 5 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('6')), 'Licensed_GenModel'] = 'BMW 6 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('7')), 'Licensed_GenModel'] = 'BMW 7 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('8')), 'Licensed_GenModel'] = 'BMW 8 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('K')), 'Licensed_GenModel'] = 'BMW K SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='BMW')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('R')), 'Licensed_GenModel'] = 'BMW R SERIES'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 1')), 'Licensed_GenModel'] = 'AUDI RS1'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 2')), 'Licensed_GenModel'] = 'AUDI RS2'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 3')), 'Licensed_GenModel'] = 'AUDI RS3'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 4')), 'Licensed_GenModel'] = 'AUDI RS4'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 5')), 'Licensed_GenModel'] = 'AUDI RS5'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 6')), 'Licensed_GenModel'] = 'AUDI RS6'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 7')), 'Licensed_GenModel'] = 'AUDI RS7'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS 8')), 'Licensed_GenModel'] = 'AUDI RS8'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS Q3')), 'Licensed_GenModel'] = 'AUDI RSQ3'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='AUDI')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RS Q8')), 'Licensed_GenModel'] = 'AUDI RSQ8'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='FIAT')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('ROLLERTEAM')), 'Licensed_GenModel'] = 'FIAT ROLLER TEAM'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='FIAT')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('GRAND PUNTO')), 'Licensed_GenModel'] = 'FIAT PUNTO'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='FIAT')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('GRANDE PUNTO')), 'Licensed_GenModel'] = 'FIAT PUNTO'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='FORD')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TRAN CONNECT')), 'Licensed_GenModel'] = 'FORD TRANSIT CONNECT'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='FORD')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TRAN CONN')), 'Licensed_GenModel'] = 'FORD TRANSIT CONNECT'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='KIA')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith("CEE'D")), 'Licensed_GenModel'] = 'KIA CEED'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='KIA')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith("PRO CEE'D")), 'Licensed_GenModel'] = 'KIA PRO CEED'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='JAGUAR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('S-TYPE')), 'Licensed_GenModel'] = 'JAGUAR S TYPE'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='JAGUAR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('X-TYPE')), 'Licensed_GenModel'] = 'JAGUAR X TYPE'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='JAGUAR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('XE')), 'Licensed_GenModel'] = 'JAGUAR XE SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='JAGUAR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('XF')), 'Licensed_GenModel'] = 'JAGUAR XF SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='JAGUAR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('XJ')), 'Licensed_GenModel'] = 'JAGUAR XJ SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='JAGUAR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('XK')), 'Licensed_GenModel'] = 'JAGUAR XK SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='JAGUAR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('XF')), 'Licensed_GenModel'] = 'JAGUAR XF SERIES'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('EVOQUE')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER EVOQUE'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('SPORT')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER SPORT'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('VELAR')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER VELAR'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('RANGE ROVER')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('RANGEROVER')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('R-ROVER')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('RROVER')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('R ROVER')), 'Licensed_GenModel'] = 'LAND ROVER RANGE ROVER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('FREELANDER')), 'Licensed_GenModel'] = 'LAND ROVER FREELANDER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('DISCOVERY')), 'Licensed_GenModel'] = 'LAND ROVER DISCOVERY'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LAND ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.contains('DISCOVRY')), 'Licensed_GenModel'] = 'LAND ROVER DISCOVERY'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RX300')), 'Licensed_GenModel'] = 'LEXUS RX 300'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RX400')), 'Licensed_GenModel'] = 'LEXUS RX 400'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RX350')), 'Licensed_GenModel'] = 'LEXUS RX 350'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RXL 450')), 'Licensed_GenModel'] = 'LEXUS RX 450'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('SC430')), 'Licensed_GenModel'] = 'LEXUS SC 430'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('LS400')), 'Licensed_GenModel'] = 'LEXUS LS 400'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('LS430')), 'Licensed_GenModel'] = 'LEXUS LS 430'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('IS200')), 'Licensed_GenModel'] = 'LEXUS IS 200'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('GS450')), 'Licensed_GenModel'] = 'LEXUS GS 450'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='LEXUS')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('GS300')), 'Licensed_GenModel'] = 'LEXUS GS 300'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MAZDA')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('RX7')), 'Licensed_GenModel'] = 'MAZDA RX-7'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ V')), 'Licensed_GenModel'] = 'MERCEDES V CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ X')), 'Licensed_GenModel'] = 'MERCEDES X CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ SLK')), 'Licensed_GenModel'] = 'MERCEDES SLK CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ SLC')), 'Licensed_GenModel'] = 'MERCEDES SLC CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ S')), 'Licensed_GenModel'] = 'MERCEDES S CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ R')), 'Licensed_GenModel'] = 'MERCEDES R CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ ML')), 'Licensed_GenModel'] = 'MERCEDES ML CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ AMG')), 'Licensed_GenModel'] = 'MERCEDES AMG CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ GLE')), 'Licensed_GenModel'] = 'MERCEDES GLE CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ GLS')), 'Licensed_GenModel'] = 'MERCEDES GLS CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ GLC')), 'Licensed_GenModel'] = 'MERCEDES GLC CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ GLA')), 'Licensed_GenModel'] = 'MERCEDES GLA CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ GLB')), 'Licensed_GenModel'] = 'MERCEDES GLB CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ GL')), 'Licensed_GenModel'] = 'MERCEDES GL CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ G')), 'Licensed_GenModel'] = 'MERCEDES G CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ E')), 'Licensed_GenModel'] = 'MERCEDES E CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('E')), 'Licensed_GenModel'] = 'MERCEDES E CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('GL')), 'Licensed_GenModel'] = 'MERCEDES GL CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ C')), 'Licensed_GenModel'] = 'MERCEDES C CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ A')), 'Licensed_GenModel'] = 'MERCEDES A CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ CLS')), 'Licensed_GenModel'] = 'MERCEDES CLS CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('CLS')), 'Licensed_GenModel'] = 'MERCEDES CLS CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('CLK')), 'Licensed_GenModel'] = 'MERCEDES CLK CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('CL')), 'Licensed_GenModel'] = 'MERCEDES CL CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('C')), 'Licensed_GenModel'] = 'MERCEDES C CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('B')), 'Licensed_GenModel'] = 'MERCEDES B CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('A')), 'Licensed_GenModel'] = 'MERCEDES A CLASS'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('-BENZ VITO')), 'Licensed_GenModel'] = 'MERCEDES VITO'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('1')), 'Licensed_GenModel'] = 'MERCEDES 190'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('2')), 'Licensed_GenModel'] = 'MERCEDES 200'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('3')), 'Licensed_GenModel'] = 'MERCEDES 300'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('4')), 'Licensed_GenModel'] = 'MERCEDES 400'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('5')), 'Licensed_GenModel'] = 'MERCEDES 500'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='MERCEDES')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('6')), 'Licensed_GenModel'] = 'MERCEDES 600'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='NISSAN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('XTRAIL')), 'Licensed_GenModel'] = 'NISSAN X-TRAIL'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='NISSAN')&(theft_years_occurences_summed_df['Make/Model'].str.contains('NV200')), 'Licensed_GenModel'] = 'NISSAN NV200'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='RENAULT')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('G-ESPACE')), 'Licensed_GenModel'] = 'RENAULT GRAND ESPACE'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='RENAULT')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('GRD ESPACE')), 'Licensed_GenModel'] = 'RENAULT GRAND ESPACE'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='RENAULT')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('GRND ESPACE')), 'Licensed_GenModel'] = 'RENAULT GRAND ESPACE'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('100')), 'Licensed_GenModel'] = 'ROVER 100 SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('111')), 'Licensed_GenModel'] = 'ROVER 100 SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('114')), 'Licensed_GenModel'] = 'ROVER 100 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('200')), 'Licensed_GenModel'] = 'ROVER 200 SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('214')), 'Licensed_GenModel'] = 'ROVER 200 SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('216')), 'Licensed_GenModel'] = 'ROVER 200 SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('218')), 'Licensed_GenModel'] = 'ROVER 200 SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('220')), 'Licensed_GenModel'] = 'ROVER 200 SERIES'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('414')), 'Licensed_GenModel'] = 'ROVER 400 SERIES'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='ROVER')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('416')), 'Licensed_GenModel'] = 'ROVER 400 SERIES'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='TOYOTA')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('HI ACE')), 'Licensed_GenModel'] = 'TOYOTA HIACE'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='TOYOTA')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('HI LUX')), 'Licensed_GenModel'] = 'TOYOTA HILUX'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='TOYOTA')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('HI-LUX')), 'Licensed_GenModel'] = 'TOYOTA HILUX'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='TOYOTA')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('LAND CRUISER')), 'Licensed_GenModel'] = 'TOYOTA LANDCRUISER'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='TVR')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('CHIMAERA')), 'Licensed_GenModel'] = 'TVR CHIMERA'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VAUXHALL')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('INSIG')), 'Licensed_GenModel'] = 'VAUXHALL INSIGNIA'


theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('T-SPORTER')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('T-PORTER')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TR-PORTER')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TPORTER')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TR-SPORTER')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TRANS')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TRANSP')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TRAN-ER')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLKSWAGEN')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('TR-PRTER')), 'Licensed_GenModel'] = 'VOLKSWAGEN TRANSPORTER'

theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']=='VOLVO')&(theft_years_occurences_summed_df['Model_Possible'].str.startswith('XC 90')), 'Licensed_GenModel'] = 'VOLVO XC90'




In [ ]:
print(len(theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']!='UNKNOWN')&(theft_years_occurences_summed_df['Year Manufacture/First Reg']>1985)]))

In [ ]:
theft_years_occurences_summed_df.loc[(theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN')&(theft_years_occurences_summed_df['Manufacturer']!='UNKNOWN')&(theft_years_occurences_summed_df['Year Manufacture/First Reg']>1985)][-3050:-3001]

In [ ]:
licensed_vehicle_df.loc[(licensed_vehicle_df['Make']=='FIAT')]['GenModel'].unique()

In [ ]:
theft_years_occurences_summed_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\theft_data_2018_2022_cleaned.csv")

In [ ]:
theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Licensed_GenModel']=='UNKNOWN']

In [ ]:
licensed_vehicle_df.loc[(licensed_vehicle_df['Make']=='BMW')]['Model'].unique()


In [ ]:
theft_years_occurences_summed_df['Model_Possible'] = theft_years_occurences_summed_df.apply(lambda x: x['Make/Model'].replace(x['Manufacturer'], '').strip(), axis = 1)

## Re-Name

In [ ]:
theft_years_summed_df = theft_years_occurences_summed_df.rename(columns={'Manufacturer':'Licensed_Manfacturer', 'Model':'Licensed_Model'})

In [ ]:
theft_years_summed_df.loc[theft_years_summed_df['Licensed_Manfacturer']=='UNKNOWN']['Make/Model'].unique()


In [ ]:
len(theft_years_summed_df.loc[theft_years_summed_df['Licensed_Manfacturer']=='UNKNOWN'])

## Formatting

In [ ]:
theft_years_summed_df.loc[(theft_years_summed_df['Licensed_Manfacturer']=='UNKNOWN'), 'Make/Model'] = theft_years_summed_df.loc[(theft_years_summed_df['Licensed_Manfacturer']=='UNKNOWN')]['Make/Model'].replace('VW', 'VOLKSWAGEN')

theft_years_summed_df.loc[(theft_years_summed_df['Licensed_Manfacturer']=='UNKNOWN'), 'Make/Model'] = theft_years_summed_df.loc[(theft_years_summed_df['Licensed_Manfacturer']=='UNKNOWN')]['Make/Model'].replace('V W', 'VOLKSWAGEN')

theft_years_summed_df['Licensed_Manfacturer'] = theft_years_summed_df['Make/Model'].apply(lambda x: extract_make_func(x))


In [ ]:
theft_years_summed_with_make_and_model_df = theft_years_summed_df.loc[(theft_years_summed_df['Licensed_Model']!='UNKNOWN')&(pd.isnull(theft_years_summed_df['Licensed_Model'])==False)]

theft_years_summed_with_make_and_model_df


- Matches been found where they shouldn't

In [ ]:
theft_years_summed_df.loc[(theft_years_summed_df['Licensed_Model']!='UNKNOWN')&(pd.isnull(theft_years_summed_df['Licensed_Model'])==False)&(theft_years_summed_df['Licensed_Manfacturer'].isin(vehicle_features_manufacturers_upper_list))]


In [ ]:
licensed_vehicle_with_trim_df = licensed_vehicle_df.copy()
licensed_vehicle_with_trim_df.insert(4, 'Trim_Engine', '')

In [ ]:
licensed_vehicle_with_trim_df['Trim_Engine'] = licensed_vehicle_with_trim_df.apply(lambda x: x['Model'].replace((x['GenModel'].replace(x['Make'], '').strip()), '').strip(), axis = 1)

In [ ]:
licensed_trims = licensed_vehicle_with_trim_df['Trim_Engine'].unique()
licensed_trims

In [ ]:
licensed_vehicle_with_trim_df

In [ ]:
licensed_vehicle_with_trim_df

## How many of each make model and trim

In [ ]:
# Full model and trim count with year
full_model_trim_count = {2022: {}, 2021: {}, 2020: {}, 2019: {}, 2018: {}}
for i, row in licensed_vehicle_with_trim_df.iterrows():
    if i % 20000 == 0:
        print(i)
    model = row["GenModel"]
    trim = row['Trim_Engine']
    for k in full_model_trim_count:
        if model not in full_model_trim_count[k].keys():
            full_model_trim_count[k][model] = {}
        if trim not in full_model_trim_count[k][model].keys():
            full_model_trim_count[k][model][trim] = 0
        if row[str(k)] != "[z]":
            full_model_trim_count[k][model][trim] += int(row[str(k)])

In [ ]:
full_model_trim_count[2022]

In [ ]:
import pickle

with open(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\licensed_vehicles_make_model_trim_dict.pkl", 'wb') as handle:
    pickle.dump(full_model_trim_count, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\licensed_vehicles_make_model_trim_dict.pkl", 'rb') as handle:
    #full_model_trim_count_loaded = pickle.load(handle)

In [ ]:
full_model_trim_count_loaded

## Count of each make Manufacturer, Model, Trim and Year

In [ ]:
licensed_vehicle_with_trim_df

In [ ]:
# Full model and trim count with year
full_model_trim_manufacture_year_count = {2022: {}, 2021: {}, 2020: {}, 2019: {}, 2018: {}}
for i, row in licensed_vehicle_with_trim_df.iterrows():
    if i % 20000 == 0:
        print(i)
    model = row["GenModel"]
    trim = row['Trim_Engine']
    manufacture_year = row['YearManufacture']
    for k in full_model_trim_manufacture_year_count:
        if model not in full_model_trim_manufacture_year_count[k].keys():
            full_model_trim_manufacture_year_count[k][model] = {}
        if trim not in full_model_trim_manufacture_year_count[k][model].keys():
            full_model_trim_manufacture_year_count[k][model][trim] = {}
        if manufacture_year not in full_model_trim_manufacture_year_count[k][model][trim].keys():
            full_model_trim_manufacture_year_count[k][model][trim][manufacture_year] = 0
        if row[str(k)] != "[z]":
            full_model_trim_manufacture_year_count[k][model][trim][manufacture_year] += int(row[str(k)])

In [ ]:
full_model_trim_manufacture_year_count

In [ ]:
import pickle

with open(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\licensed_vehicles_make_model_trim_manufactureYear_dict.pkl", 'wb') as handle:
    pickle.dump(full_model_trim_manufacture_year_count, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
licensed_vehicle_df

In [ ]:
licensed_vehicle_df

In [ ]:
theft_years_summed_with_make_and_model_df

In [ ]:
full_model_trim_manufacture_year_count

In [ ]:
theft_years_summed_with_make_and_model_df['Trim_Engine'] = theft_years_summed_with_make_and_model_df.apply(lambda x: x['Make/Model'].replace(x['Licensed_Manfacturer'], '').replace(x['Licensed_Model'], '').strip(), axis=1)

In [ ]:
theft_years_summed_with_make_and_model_df

- How many trims match?

In [ ]:
theft_years_summed_with_make_and_model_df.loc[theft_years_summed_with_make_and_model_df['Licensed_Manfacturer']=='AUDI']['Trim_Engine'].unique()

In [ ]:
licensed_vehicle_with_trim_df.loc[licensed_vehicle_with_trim_df['Make']=='AUDI']['Trim_Engine'].unique()


In [ ]:
theft_trims = theft_years_summed_with_make_and_model_df['Trim_Engine'].unique()
theft_trims

In [ ]:
len(theft_trims)

In [ ]:
trims_exact_match = 0
for val in theft_trims:
    if val in licensed_trims:
        trims_exact_match+=1
trims_exact_match

- Josh's approach

In [ ]:
models = list(set(licensed_vehicle_df["GenModel"]))
matched = {}
x = 0
for i in range(len(models)):
    if i % 100 == 0:
        print(i)
    matched[models[i]] = list(theft_years_summed_df[theft_years_summed_df["Make/Model"].apply(lambda x: models[i].upper() in x.upper())]["Make/Model"])
    

In [ ]:
matched

In [ ]:
tally_licensed=0

tally_theft_make_model = 0

for val in matched:
    if matched[val]!=[]:
        tally_licensed+=1
        tally_theft_make_model+=len(matched[val])
print(tally_licensed)
print(tally_theft_make_model)

In [ ]:
theft_years_summed_df.loc[((theft_years_summed_df['Licensed_Model']=='UNKNOWN')|(pd.isnull(theft_years_summed_df['Licensed_Model'])==True))&(theft_years_summed_df['Licensed_Manfacturer']!='UNKNOWN')&(theft_years_summed_df['Licensed_Manfacturer'].isin(vehicle_features_manufacturers_upper_list))]['Make/Model'].unique()

In [ ]:
theft_years_summed_df.loc[((theft_years_summed_df['Licensed_Model']=='UNKNOWN')|(pd.isnull(theft_years_summed_df['Licensed_Model'])==True))&(theft_years_summed_df['Licensed_Manfacturer']!='UNKNOWN')&(theft_years_summed_df['Licensed_Manfacturer'].isin(vehicle_features_manufacturers_upper_list))]

In [ ]:
current_vehicle_file_df.loc[current_vehicle_file_df['Manufacturer']=='AC']

## Looking at trim

In [ ]:
GenModel_found_df = theft_years_occurences_summed_df.loc[theft_years_occurences_summed_df['Licensed_GenModel']!='UNKNOWN']

GenModel_found_df

In [ ]:
GenModel_found_df['Trim_Engine'] = GenModel_found_df.apply(lambda x: x['Make/Model'].replace(x['Licensed_GenModel'], ''), axis = 1)

In [ ]:
GenModel_found_df

In [ ]:
GenModel_found_df['Licensed_Trim_Engine'] = ''

In [ ]:
GenModel_found_df.loc[GenModel_found_df['Manufacturer']=='AUDI']

In [ ]:
GenModel_found_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\theft_data_trim_split_2018_2022_cleaned.csv")

In [ ]:
licensed_vehicle_df.head()

In [ ]:
def get_licensed_trim(manufacturer, gen_model, full_model):
    #manufacturer = x['Make']
    #gen_model = x['GenModel']
    #full_model = x['Model']
    
    gen_model_make_removed = gen_model.replace(manufacturer, '').strip()
    
    trim = full_model.replace(gen_model_make_removed, '').strip()
    return trim

licensed_vehicle_df['Licensed_Trim'] = licensed_vehicle_df.apply(lambda x: get_licensed_trim(x['Make'], x['GenModel'], x['Model']), axis = 1)

In [ ]:
licensed_vehicle_df

In [ ]:
licensed_make_model_trim = {}
for manufacturer_val in licensed_manufacturers:
    manufacturer_dict = {manufacturer_val : {}}
    licensed_make_model_trim = {**licensed_make_model_trim, **manufacturer_dict}
    gen_models = list(licensed_vehicle_df.loc[licensed_vehicle_df['Make'] == manufacturer_val]['GenModel'].unique())
    for gen_model_val in gen_models:
        trim_engines = list(licensed_vehicle_df.loc[(licensed_vehicle_df['Make'] == manufacturer_val)&(licensed_vehicle_df['GenModel'] == gen_model_val)]['Licensed_Trim'].unique())
        licensed_make_model_trim[manufacturer_val][gen_model_val] = trim_engines
#licensed_vehicle_df

In [ ]:
licensed_make_model_trim

- Saving output (dictionary) as a pickle file

In [ ]:
import pickle

In [ ]:
output = open(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\licensed_make_genModel_trim.pkl", 'wb')
pickle.dump(licensed_make_model_trim, output)
output.close()

# read python dict back from the file
pkl_file = open(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\licensed_make_genModel_trim.pkl", 'rb')
licensed_make_model_trim_loaded = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
GenModel_found_df

- Corrections

In [ ]:
GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel']=='ACCESS 450', 'Manufacturer'] = 'ACCESS'
GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel'].str.contains('ACCESS'), 'Manufacturer'] = 'ACCESS'
GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel'].str.contains('ALFA ROMEO'), 'Manufacturer'] = 'ALFA ROMEO'

GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel']=='ALFA ROMEO33', 'Licensed_GenModel'] = 'ALFA ROMEO 33'


In [ ]:
GenModel_found_df['Manufacturer'] = GenModel_found_df['Manufacturer'].apply(lambda x: x.strip())
GenModel_found_df['Licensed_GenModel'] = GenModel_found_df['Licensed_GenModel'].apply(lambda x: x.strip())

In [ ]:
licensed_manufacturers = list(licensed_manufacturers)
licensed_manufacturers.sort(key=len, reverse=True)
licensed_manufacturers[:5]

In [ ]:
def check_licences_manufacturers(manufacturer, make_model):
    for val in licensed_manufacturers:
        if val in make_model:
            correct_manufacturer = val
            break
        else:
            correct_manufacturer = manufacturer
    return correct_manufacturer

GenModel_found_df['Manufacturer'] = GenModel_found_df.apply(lambda x: check_licences_manufacturers(x['Manufacturer'], x['Make/Model']), axis = 1)


In [ ]:
GenModel_found_df = GenModel_found_df.drop(GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel']=='AJSDESERT SCRAMBLER'].index)
GenModel_found_df = GenModel_found_df.drop(GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel']=='AJS NAC 12'].index)

In [ ]:
GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel']=='ABARTH 595']

In [ ]:
licensed_vehicle_df.loc[licensed_vehicle_df['GenModel']=='ABARTH 595']#['GenModel'].unique()

## Searching for Licensed trims in theft data trims

In [ ]:
GenModel_found_df.loc[GenModel_found_df['Licensed_GenModel']=='KIA RIO']

In [ ]:
licensed_vehicle_df.loc[licensed_vehicle_df['GenModel']=='FORD FIESTA']['Make'].unique()

In [ ]:
def search_licensed_trim(manufacturer, gen_model, trim_engine):
    if (len(licensed_vehicle_df.loc[licensed_vehicle_df['Make']==manufacturer])>0) & (len(licensed_vehicle_df.loc[(licensed_vehicle_df['Make']==manufacturer)&(licensed_vehicle_df['GenModel']==gen_model)])>0):
        trims_list = licensed_make_model_trim[manufacturer][gen_model]
        
        trims_list.sort(key=len, reverse=True)
        #print(trims_list)
        #print(trim_engine)
        for trim in trims_list:
            if trim.strip() == trim_engine.strip():
                licensed_trim_engine = trim.strip()
                #print(trim.strip())
                #print(trim_engine.strip())
                break
                
            elif trim.strip() in trim_engine.strip():
                licensed_trim_engine = trim.strip()
                #print(trim.strip())
                #print(trim_engine.strip())
                break
                
            else:
                licensed_trim_engine = ''
    else:
        licensed_trim_engine = ''
    
    return licensed_trim_engine

In [ ]:
GenModel_found_df['Licensed_Trim_Engine'] = GenModel_found_df.apply(lambda x: search_licensed_trim(x['Manufacturer'],x['Licensed_GenModel'], x['Trim_Engine']), axis = 1)

In [ ]:
GenModel_found_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\03_Cleaned_Data_Sources\DVLA\theft_data_trim_split_2018_2022_cleaned.csv")

In [ ]:
GenModel_found_df['Licensed_Trim_Engine'].unique()

In [ ]:
GenModel_found_df.loc[(GenModel_found_df['Licensed_Trim_Engine']!="")&(pd.isnull(GenModel_found_df['Licensed_Trim_Engine'])==False)&(GenModel_found_df['Manufacturer']=='AUDI')]

In [ ]:
licensed_vehicle_df

In [ ]:
GenModel_found_df.loc[GenModel_found_df['Manufacturer']=='BMW']

In [ ]:
licensed_vehicle_df.loc[(licensed_vehicle_df['Make']=='BMW')]['Model'].unique()#&(licensed_vehicle_df['Model']=='114D ES')]#['Model'].unique()

- Partial matching...

In [ ]:
missing_trim_match = GenModel_found_df.loc[(GenModel_found_df['Licensed_Trim_Engine']=="")|(pd.isnull(GenModel_found_df['Licensed_Trim_Engine'])==True)]
missing_trim_match

In [ ]:
licensed_make_model_trim['ABARTH']

In [ ]:
from fuzzywuzzy import process

In [ ]:
def search_licensed_trim_partial(manufacturer, gen_model, trim_engine):
    if (len(licensed_vehicle_df.loc[licensed_vehicle_df['Make']==manufacturer])>0) & (len(licensed_vehicle_df.loc[(licensed_vehicle_df['Make']==manufacturer)&(licensed_vehicle_df['GenModel']==gen_model)])>0):
        trims_list = licensed_make_model_trim[manufacturer][gen_model]
        
        trims_list.sort(key=len, reverse=True)
        
        trims_list = [trim_val.strip() for trim_val in trims_list]
        
        trim_engine = trim_engine.strip()
        
        results = process.extract(trim_engine, trims_list)
        
        print(results)
            
    
    return results

In [ ]:
theft_df

In [ ]:
missing_trim_match.loc[missing_trim_match['Trim_Engine']==""]

In [ ]:
missing_trim_match['Licensed_Trim_Engine'] = missing_trim_match.apply(lambda x: search_licensed_trim_partial(x['Manufacturer'],x['Licensed_GenModel'], x['Trim_Engine']), axis = 1)

In [ ]:
missing_trim_match['Manufacturer'].unique()

In [ ]:
fuzz.partial_ratio('SPIDER MULTIAIR', '124 SPIDER MULTIAIR')

In [ ]:
for index, row in theft_years_summed_df.iterrows():
    make_model = row['Make/Model']
    
    if make_model in gen_models_unique_list:
        theft_years_summed_df.loc[index, 'Exact_Match'] = 1
        
    """    
    else:
        matches = []
        scores=[]
        for val in gen_models_unique_list:
            matches.append(val)
            score = fuzz.token_sort_ratio(val, make_model)
            scores.append(score)
            
        index_best_match = scores.index(np.max(scores))
        best_match = matches[index_best_match]
        
        theft_years_summed_df.loc[index, 'Match_Score'] = np.max(scores)
        
        theft_years_summed_df.loc[index, 'Possible_Match'] = best_match
    """

In [ ]:
theft_years_summed_df.loc[theft_years_summed_df['Exact_Match']==1]

In [ ]:
matches = 0
for year in full_model_count.keys():
    print(year)
    models_with_frequency = full_model_count[year]
    for make_model in models_with_frequency:
        vehicles_matched_df = theft_df.loc[theft_df['Make/Model']==make_model]
        if len(vehicles_matched_df)>0:
            matches+=1
matches

In [ ]:
theft_proportions = {}
for i, row in casualty_df.iterrows():
    if i % 20000 == 0:
        print(i)
    ref = row["accident_reference"]
    if ref not in casualties.keys():
        casualties[ref] = []
    casualties[ref].append(row)
print(len(casualties))

In [ ]:
# Calculate theft values
theft_values = {}
#  Max proportion
#  Average proportion of vehicles stolen across years

## Look at trends seen across the year span for vehicle make/models

    - Maybe apply the linear gradient to scale up older/newer vehicles depending on their theft liklihood.

In [ ]:
theft_values_df = pd.DataFrame(columns={"GenModel","Theft_rating"})
theft_values_df = theft_values_df.assign(GenModel=theft_values.keys(), Theft_rating=theft_values.values())

## Overlays to apply after frequency analysis is performed

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
current_vehicle_df = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\08_Master_File\vehicle_features_Vehicle_File_Master_05_09_2023.csv")

In [ ]:
current_vehicle_df

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X = current_vehicle_df.select_dtypes(include=[np.number])
X

- Security Thatcham

In [ ]:
security_thatcham_dict = {'T1':1, 'T2':2, 'N1':3, 'N2':4}

In [ ]:
security_thatcham_numeric = current_vehicle_df.loc[:,['Security_Thatcham', 'ABI_Code_44']]
security_thatcham_numeric.loc[:,'Security_Thatcham_Numeric'] = security_thatcham_numeric['Security_Thatcham'].apply(lambda x: security_thatcham_dict[x] if pd.isnull(x)==False else x)

In [ ]:
security_thatcham_numeric['Security_Thatcham_Numeric'].unique()

- Fuel_Desc

In [ ]:
current_vehicle_df['Fuel_Desc'].unique()

In [ ]:
fuel_desc_dict = {'Electric':1, 'Plug-in Hybrid':2, 'Regen Hybrid':3, 'Other':4, 'Petrol BiFuel':5, 'Diesel':6, 'Petrol':7}

fuel_desc_numeric = current_vehicle_df.loc[:,['Fuel_Desc', 'ABI_Code_44']]
fuel_desc_numeric.loc[:,'Fuel_Desc_Numeric'] = fuel_desc_numeric['Fuel_Desc'].apply(lambda x: fuel_desc_dict[x] if pd.isnull(x)==False else x)

## Merging numeric converted columns

In [ ]:
X = X.merge(security_thatcham_numeric, on = ['ABI_Code_44', 'ABI_Code_44'], how = 'left')

In [ ]:
X = X.merge(fuel_desc_numeric, on = ['ABI_Code_44', 'ABI_Code_44'], how = 'left')


- Drop columns not needed

In [ ]:
X_Copy = X.copy()

In [ ]:
X = X.drop(columns=['Security_Thatcham', 'Fuel_Desc', 'vehicle_features_MR_Risk_Theft_23', 'ABI_Code_44'])

In [ ]:
columns_list = list(X.columns)
columns_list.remove('Bumper_Score_RCAR')
columns_list[:5]

In [ ]:
len(columns_list)

- Replacing -1 placeholders

In [ ]:
X[X['End_Year_Veh_Generation']==-1]

In [ ]:
for col_header in columns_list:
    X.loc[X[col_header]==-1, col_header] = np.full(len(X.loc[X[col_header]==-1]), np.nan)

In [ ]:
X

In [ ]:
y = current_vehicle_df['vehicle_features_MR_Risk_Theft_23']
y

In [ ]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_train

In [ ]:
bst = XGBRegressor(n_estimators=200, eta= 0.1, max_depth=15)
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(preds, y_test)
mse

In [ ]:
bst.feature_importances_

In [ ]:
X_train

In [ ]:
from xgboost import plot_importance

plot_importance(bst, max_num_features=20)

# Uplift to theft ratings

- Start_Year_Veh_Generation

- Safety_Manufacturer

- Volume_Box

- Engine_CC_Petro_Electric

- Body_Score_NVSA

- ID_Score_NVSA

- Electronic_Score_NVSA

- Security_Thatcham_Numeric

Bring in price

How to combine...

### Looking at price...

In [ ]:
vehicle_file_price_df = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\02_Vehicle_File_Collaboration\vehicle_features_Vehicle_File_Best_Matched_With_Local_Fills_Ben_New.csv")

vehicle_file_price_df = vehicle_file_price_df.drop(columns=['Unnamed: 0'])

In [ ]:
vehicle_price_original_df = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\02_Vehicle_File_Collaboration\vehicle_features_Vehicle_File_Best_Matched_With_Local_Fills_Ben_New.csv")

In [ ]:
print(vehicle_price_original_df['Price_New'].isna().sum())

print(vehicle_price_original_df['Price_Used_Average'].isna().sum())

In [ ]:
vehicle_file_price_df['Manufacturer'].unique()

In [ ]:
grouping_less_granular_original_df = vehicle_file_price_df.groupby(by=['Manufacturer', 'Model', 'Veh_Generation', 'Start_Year_Veh_Generation', 'Veh_Desc_ABI','Fuel_Primary', 'Fuel_Secondary', 'Engine_Size_Litres', 'Body_Type', 'Aspiration'], dropna=False).agg({'ABI_Code':'count', 'Price_New':'mean', 'Price_Used_Average':'mean'}).reset_index()

grouping_less_granular_original_df

- Median values

In [ ]:
grouping_less_granular_original_median_df = vehicle_file_price_df.groupby(by=['Manufacturer', 'Model', 'Veh_Generation', 'Start_Year_Veh_Generation', 'Veh_Desc_ABI','Fuel_Primary', 'Fuel_Secondary', 'Engine_Size_Litres', 'Body_Type', 'Aspiration'], dropna=False).agg({'ABI_Code':'count', 'Price_New':'median', 'Price_Used_Average':'median'}).reset_index()

In [ ]:
grouping_less_granular_original_median_df.loc[grouping_less_granular_original_median_df['Price_Used_Average']>grouping_less_granular_original_median_df['Price_New']]

In [ ]:
grouping_less_granular_original_df['Price_New'].isna().sum()

In [ ]:
grouping_less_granular_original_df.loc[pd.isnull(grouping_less_granular_original_df['Price_New'])==True]

- Thatcham Price data

In [ ]:
import pandas as pd

In [ ]:
thatcham_df = pd.read_excel(r"C:\Users\benwa\vehicle_features\vehicle_features FS - Documents\09_Data\Thatcham\2023_20\vehicle_risk_data_advanced_Car_full_2023.20.xlsx")

In [ ]:
for val in thatcham_df.columns:
    if 'abi' in val.lower():
        print(val)

In [ ]:
thatcham_df[['TVI Code', 'ABI Description', 'List Price']].groupby(by=['ABI Description']).agg({'List Price':'mean'}).reset_index()

In [ ]:
thatcham_grouped_df = thatcham_df.groupby(by = ['ABI Description'], dropna=False).agg({'List Price' : 'mean'}).reset_index()

thatcham_grouped_df

In [ ]:
thatcham_grouped_df = thatcham_grouped_df.rename(columns={'ABI Description' : 'Veh_Desc_ABI'})

In [ ]:
thatcham_grouped_df

- Merging on Thatcham data

In [ ]:
grouping_less_granular_original_df = grouping_less_granular_original_df.merge(thatcham_grouped_df[['Veh_Desc_ABI', 'List Price']], on =['Veh_Desc_ABI', 'Veh_Desc_ABI'], how = 'left')
grouping_less_granular_original_df

In [ ]:
grouping_less_granular_original_df.loc[grouping_less_granular_original_df['Price_New']!=grouping_less_granular_original_df['List Price']]

In [ ]:
grouping_less_granular_original_df.loc[(pd.isnull(grouping_less_granular_original_df['Price_New'])==True)&(pd.isnull(grouping_less_granular_original_df['List Price'])==False), 'Price_New'] = grouping_less_granular_original_df.loc[(pd.isnull(grouping_less_granular_original_df['Price_New'])==True)&(pd.isnull(grouping_less_granular_original_df['List Price'])==False)]['List Price']

- Automate with closest match

In [ ]:
grouping_less_granular_original_df.loc[pd.isnull(grouping_less_granular_original_df['Price_New'])==True]

In [ ]:
grouping_less_granular_original_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\grouped_price_data_original.csv")

In [ ]:
abi_desc_missing = list(grouping_less_granular_original_df.loc[pd.isnull(grouping_less_granular_original_df['Price_New'])==True]['Veh_Desc_ABI'].unique())

In [ ]:
len(grouping_less_granular_original_df)

In [ ]:
grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==False)&(grouping_less_granular_df['Veh_Desc_ABI'].isin(abi_desc_missing))][::100]


In [ ]:
grouping_less_granular_df = grouping_less_granular_original_df.copy()

In [ ]:
no_matches = 0
no_value_with_match = 0

for index, row in grouping_less_granular_df.loc[pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True].iterrows():
    manufacturer = row['Manufacturer']
    model = row['Model']
    generation = row['Veh_Generation']
    primary_fuel = row['Fuel_Primary']
    fuel_secondary = row['Fuel_Secondary']
    body_type = row['Body_Type']
    engine_size_litres = row['Engine_Size_Litres']
    aspiration = row['Aspiration']
    abi_desc = row['Veh_Desc_ABI']
    start_year = row['Start_Year_Veh_Generation']
    
    
    
    subset_price_1_df = grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']==manufacturer)&(grouping_less_granular_df['Model']==model)]#&(grouping_less_granular_df['Engine_Size_Litres']==engine_size_litres)]#((grouping_less_granular_df['Start_Year_Veh_Generation']==start_year)|(grouping_less_granular_df['Start_Year_Veh_Generation']==start_year+1)|(grouping_less_granular_df['Start_Year_Veh_Generation']==start_year-1))&(grouping_less_granular_df['Aspiration']==aspiration)]
    
    if len(subset_price_1_df)>0:
        mean_price_estimate = subset_price_1_df['Price_New_Estimate'].mean()
        
        if pd.isnull(mean_price_estimate)==False:
            grouping_less_granular_df.loc[index, 'Price_New_Estimate'] = mean_price_estimate
        else:
            no_value_with_match+=1
    else:
        no_matches+=1
    
print(no_matches)
print(no_value_with_match)
            
"""
subset_price_2_df = grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']==manufacturer)&(grouping_less_granular_df['Model']==model)&(grouping_less_granular_df['Fuel_Primary']==primary_fuel)&(grouping_less_granular_df['Body_Type']==body_type)&(grouping_less_granular_df['Fuel_Secondary']==fuel_secondary)]
if len(subset_price_2_df)>0:
    mean_price_estimate = subset_price_2_df['Price_New_Estimate'].mean()
    
    if pd.isnull(mean_price_estimate) == False:
        second_level_match+=1
        grouping_less_granular_df.loc[index, 'Price_New_Estimate'] = mean_price_estimate
    else:
        subset_price_3_df = grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']==manufacturer)&(grouping_less_granular_df['Model']==model)&(grouping_less_granular_df['Fuel_Primary']==primary_fuel)&(grouping_less_granular_df['Body_Type']==body_type)]
        if len(subset_price_3_df)>0:
            mean_price_estimate = subset_price_3_df['Price_New_Estimate'].mean()
            if pd.isnull(mean_price_estimate)==False:
                third_level_match+=1
                grouping_less_granular_df.loc[index, 'Price_New_Estimate'] = mean_price_estimate
            
            else:
                subset_price_4_df = grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']==manufacturer)&(grouping_less_granular_df['Model']==model)&(grouping_less_granular_df['Fuel_Primary']==primary_fuel)&(grouping_less_granular_df['Body_Type']==body_type)]
                

                if len(subset_price_4_df)>0:
                    mean_price_estimate = subset_price_4_df['Price_New_Estimate'].mean()
                    if pd.isnull(mean_price_estimate)==False:
                        grouping_less_granular_df.loc[index, 'Price_New_Estimate'] = mean_price_estimate
                    else:
                        subset_price_5_df = grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']==manufacturer)&(grouping_less_granular_df['Model']==model)&(grouping_less_granular_df['Fuel_Primary']==primary_fuel)]
                        if len(subset_price_5_df)>0:
                            mean_price_estimate = subset_price_5_df['Price_New_Estimate'].mean()
                            if pd.isnull(mean_price_estimate) == False:
                                grouping_less_granular_df.loc[index, 'Price_New_Estimate'] = mean_price_estimate
                            else:
                                subset_price_6_df = grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']==manufacturer)&(grouping_less_granular_df['Model']==model)&(grouping_less_granular_df['Veh_Generation']==generation)]
                                if len(subset_price_6_df)>0:
                                    mean_price_estimate = subset_price_6_df['Price_New_Estimate'].mean()
                                    if pd.isnull(mean_price_estimate) == False:
                                        grouping_less_granular_df.loc[index, 'Price_New_Estimate'] = mean_price_estimate
"""



In [ ]:
grouping_less_granular_df['Price_New_Estimate'].isna().sum()

- search for these missing values

In [ ]:
grouping_less_granular_df.loc[pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True][:40]

In [ ]:
grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Chevrolet')&(grouping_less_granular_df['Model']=='Chevelle')]

In [ ]:
grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='AC')&(grouping_less_granular_df['Model']=='Ace')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 90000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='AC')&(grouping_less_granular_df['Model']=='Cobra')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 85000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='400')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 7200

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='500-4')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 7200

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='500-5')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 7200

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='Mega')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 12000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='A751')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 7400

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='700')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 7400

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='City')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 10000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aixam')&(grouping_less_granular_df['Model']=='GTO')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 10000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Alfa Romeo')&(grouping_less_granular_df['Model']=='8C Spider')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 174000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Alfa Romeo')&(grouping_less_granular_df['Model']=='8C')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 111000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Alfa Romeo')&(grouping_less_granular_df['Model']=='SZ')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 42500

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Alpina')&(grouping_less_granular_df['Model']=='B5')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 70000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Asia')&(grouping_less_granular_df['Model']=='Rocsta')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 9950

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Bugatti')&(grouping_less_granular_df['Model']=='Veyron')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 1000000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Bugatti')&(grouping_less_granular_df['Model']=='Chiron')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 1900000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Bugatti')&(grouping_less_granular_df['Model']=='Divo')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 4200000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Buick')&(grouping_less_granular_df['Model']=='Regal')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 25000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Aston Martin')&(grouping_less_granular_df['Model']=='One-77')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 1400000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Caterham')&(grouping_less_granular_df['Model']=='Seven')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 30000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Chrysler')&(grouping_less_granular_df['Model']=='Crossfire')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 26000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Dodge')&(grouping_less_granular_df['Model']=='Viper')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 70000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Chrysler')&(grouping_less_granular_df['Model']=='Viper')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 70000

grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Dodge')&(grouping_less_granular_df['Model']=='SRT10')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 77500

#grouping_less_granular_df.loc[(grouping_less_granular_df['Manufacturer']=='Chevrolet')&(grouping_less_granular_df['Model']=='Caprice')&(pd.isnull(grouping_less_granular_df['Price_New_Estimate'])==True), 'Price_New_Estimate'] = 4000



In [ ]:
list(grouping_less_granular_df.columns)

In [ ]:
grouping_less_granular_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\grouped_price_data_new_price_filled.csv")

In [ ]:
grouping_less_granular_mean_df = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\grouped_price_data_new_price_filled.csv")

grouping_less_granular_mean_df

- Separate out:

    - High performance (Super/Hyper cars), High value cars, Classic cars

In [ ]:
grouping_less_granular_df['Manufacturer'].unique()

In [ ]:
grouping_less_granular_df.loc[pd.isnull(grouping_less_granular_df['Price_New'])==True]

In [ ]:
grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Aston Martin')&(grouping_less_granular_df['Model']=='Valkyrie'), 'Price_New'] = 2500000
grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_Used_Average'])==True)&(grouping_less_granular_df['Manufacturer']=='Aston Martin')&(grouping_less_granular_df['Model']=='Valkyrie'), 'Price_Used_Average'] = 3500000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='BMW')&(grouping_less_granular_df['Model']=='ActiveHybrid 5'), 'Price_New'] = 46860

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Bentley')&(grouping_less_granular_df['Model']=='Bentayga PHEV'), 'Price_New'] = 133100

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Cadillac')&(grouping_less_granular_df['Model']=='BLS'), 'Price_New'] = 19950

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Cadillac')&(grouping_less_granular_df['Model']=='BLS')&(grouping_less_granular_df['Body_Type']=='Saloon'), 'Price_Used_Average'] = 1000
grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_Used_Average'])==True)&(grouping_less_granular_df['Manufacturer']=='Cadillac')&(grouping_less_granular_df['Model']=='BLS')&(grouping_less_granular_df['Body_Type']=='Estate'), 'Price_Used_Average'] = 2400

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Ferrari')&(grouping_less_granular_df['Model']=='296')&(grouping_less_granular_df['Veh_Desc_ABI']=='296 GTB'), 'Price_New'] = 241550
grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_Used_Average'])==True)&(grouping_less_granular_df['Manufacturer']=='Ferrari')&(grouping_less_granular_df['Model']=='296')&(grouping_less_granular_df['Veh_Desc_ABI']=='296 GTB'), 'Price_Used_Average'] = 230000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Ferrari')&(grouping_less_granular_df['Model']=='296')&(grouping_less_granular_df['Veh_Desc_ABI']=='296 GTS'), 'Price_New'] = 279000
grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_Used_Average'])==True)&(grouping_less_granular_df['Manufacturer']=='Ferrari')&(grouping_less_granular_df['Model']=='296')&(grouping_less_granular_df['Veh_Desc_ABI']=='296 GTS'), 'Price_Used_Average'] = 270000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Ferrari')&(grouping_less_granular_df['Model']=='SF90'), 'Price_New'] = 376000
grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_Used_Average'])==True)&(grouping_less_granular_df['Manufacturer']=='Ferrari')&(grouping_less_granular_df['Model']=='SF90'), 'Price_Used_Average'] = 370000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='SEAT')&(grouping_less_granular_df['Model']=='Leon'), 'Price_New'] = 18500

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Honda')&(grouping_less_granular_df['Model']=='Fit'), 'Price_New'] = 13000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Honda')&(grouping_less_granular_df['Model']=='Insight'), 'Price_New'] = 17170

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Volvo')&(grouping_less_granular_df['Model']=='V70'), 'Price_New'] = 28985

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Volvo')&(grouping_less_granular_df['Model']=='V50'), 'Price_New'] = 19325

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Volvo')&(grouping_less_granular_df['Model']=='S80'), 'Price_New'] = 19325

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Volvo')&(grouping_less_granular_df['Model']=='S40'), 'Price_New'] = 16000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Volkswagen')&(grouping_less_granular_df['Model']=='Touareg Hybrid'), 'Price_New'] = 57500

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Toyota')&(grouping_less_granular_df['Model']=='Prius'), 'Price_New'] = 21210

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Toyota')&(grouping_less_granular_df['Model']=='Auris Hybrid')&(grouping_less_granular_df['Veh_Desc_ABI']=='Auris Hybrid T4'), 'Price_New'] = 18950

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Toyota')&(grouping_less_granular_df['Model']=='Estima'), 'Price_New'] = 32000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Toyota')&(grouping_less_granular_df['Model']=='Noah'), 'Price_New'] = 34000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Toyota')&(grouping_less_granular_df['Model']=='Auris Hybrid')&(grouping_less_granular_df['Veh_Desc_ABI']=='Auris Hybrid T-Spirit'), 'Price_New'] = 20700

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Saab')&(grouping_less_granular_df['Model']=='9-5'), 'Price_New'] = 25500

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Saab')&(grouping_less_granular_df['Model']=='9-3'), 'Price_New'] = 26265

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Renault')&(grouping_less_granular_df['Model']=='Clio'), 'Price_New'] = 11175

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Koenigsegg')&(grouping_less_granular_df['Model']=='Regera'), 'Price_New'] = 1400000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Land Rover')&(grouping_less_granular_df['Model']=='Range Rover Hybrid'), 'Price_New'] = 100000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Porsche')&(grouping_less_granular_df['Model']=='918 PHEV'), 'Price_New'] = 678000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Porsche')&(grouping_less_granular_df['Model']=='Cayenne Hybrid'), 'Price_New'] = 50000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Porsche')&(grouping_less_granular_df['Model']=='Panamera Hybrid'), 'Price_New'] = 86500

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Lexus')&(grouping_less_granular_df['Model']=='GS Hybrid'), 'Price_New'] = 46690

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Lexus')&(grouping_less_granular_df['Model']=='RX Hybrid'), 'Price_New'] = 54000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='McLaren')&(grouping_less_granular_df['Model']=='Artura'), 'Price_New'] = 189200

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='McLaren')&(grouping_less_granular_df['Model']=='Speedtail'), 'Price_New'] = 1750000

grouping_less_granular_df.loc[(pd.isnull(grouping_less_granular_df['Price_New'])==True)&(grouping_less_granular_df['Manufacturer']=='Mitsubishi')&(grouping_less_granular_df['Model']=='Outlander PHEV'), 'Price_New'] = 35805


In [ ]:
grouping_less_granular_df = grouping_less_granular_df.rename(columns={'Price_New':'Price_New_Estimate', 'Price_Used_Average':'Price_Used_Average_Estimate'})
grouping_less_granular_df.head()

In [ ]:
len(grouping_less_granular_df)

- Join back to overall df

    - Master File

In [ ]:
price_new_estimate_dict = grouping_less_granular_df.set_index('Veh_Desc_ABI')['Price_New_Estimate'].to_dict()
price_used_estimate_dict = grouping_less_granular_df.set_index('Veh_Desc_ABI')['Price_Used_Average_Estimate'].to_dict()

price_new_estimate_dict

In [ ]:
grouping_less_granular_df['Price_New_Estimate'].isna().sum()

In [ ]:
import numpy as np

In [ ]:
master_file = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\08_Master_File\vehicle_features_Vehicle_File_Master_05_09_2023.csv")


- Merge two data frames on

In [ ]:
grouping_less_granular_df = grouping_less_granular_df.rename(columns = {'Price_New_Estimate':'Price_New_Estimate_Filled', 'Price_Used_Average_Estimate':'Price_Used_Mean_Estimate'})

In [ ]:
grouping_less_granular_original_median_df = grouping_less_granular_original_median_df.rename(columns = {'Price_New':'Price_New_Median', 'Price_Used_Average':'Price_Used_Median'})


In [ ]:
vehicle_file_price_prediction_df = master_file.copy()

vehicle_file_price_prediction_df = vehicle_file_price_prediction_df.merge(grouping_less_granular_df[['Manufacturer',
                                                                                                    'Model',
                                                                                                    'Veh_Generation',
                                                                                                    'Start_Year_Veh_Generation',
                                                                                                    'Veh_Desc_ABI',
                                                                                                    'Fuel_Primary',
                                                                                                    'Fuel_Secondary',
                                                                                                    'Engine_Size_Litres',
                                                                                                    'Body_Type',
                                                                                                    'Aspiration', 'Price_New_Estimate_Filled', 'Price_Used_Mean_Estimate']], 
                                                                                                    on = ['Manufacturer',
                                                                                                    'Model',
                                                                                                    'Veh_Generation',
                                                                                                    'Start_Year_Veh_Generation',
                                                                                                    'Veh_Desc_ABI',
                                                                                                    'Fuel_Primary',
                                                                                                    'Fuel_Secondary',
                                                                                                    'Engine_Size_Litres',
                                                                                                    'Body_Type',
                                                                                                    'Aspiration'], 
                                                                                                     how = 'left')


In [ ]:

vehicle_file_price_prediction_df = vehicle_file_price_prediction_df.merge(grouping_less_granular_original_median_df[['Manufacturer',
                                                                                                    'Model',
                                                                                                    'Veh_Generation',
                                                                                                    'Start_Year_Veh_Generation',
                                                                                                    'Veh_Desc_ABI',
                                                                                                    'Fuel_Primary',
                                                                                                    'Fuel_Secondary',
                                                                                                    'Engine_Size_Litres',
                                                                                                    'Body_Type',
                                                                                                    'Aspiration', 'Price_New_Median', 'Price_Used_Median']], 
                                                                                                    on = ['Manufacturer',
                                                                                                    'Model',
                                                                                                    'Veh_Generation',
                                                                                                    'Start_Year_Veh_Generation',
                                                                                                    'Veh_Desc_ABI',
                                                                                                    'Fuel_Primary',
                                                                                                    'Fuel_Secondary',
                                                                                                    'Engine_Size_Litres',
                                                                                                    'Body_Type',
                                                                                                    'Aspiration'], 
                                                                                                     how = 'left')


In [ ]:
vehicle_file_price_prediction_df

In [ ]:
vehicle_file_price_prediction_df['Price_New_Final_Filled'] = vehicle_file_price_prediction_df.apply(lambda x: x['Price_New_Median'] if pd.isnull(x['Price_New_Median'])==False else x['Price_New_Estimate_Filled'], axis = 1)

vehicle_file_price_prediction_df['Price_Used_Final_Filled'] = vehicle_file_price_prediction_df.apply(lambda x: x['Price_Used_Median'] if pd.isnull(x['Price_Used_Median'])==False else x['Price_Used_Mean_Estimate'], axis = 1)

vehicle_file_price_prediction_df.loc[pd.isnull(vehicle_file_price_prediction_df['Price_New_Final_Filled'])==True, 'Price_New_Final_Filled'] = vehicle_file_price_prediction_df.loc[pd.isnull(vehicle_file_price_prediction_df['Price_New_Final_Filled'])==True].apply(lambda x: price_vals[x['ABI_Code_44']] if x['ABI_Code_44'] in (price_vals.keys()) else x['Price_New_Final_Filled'], axis = 1)

In [ ]:
vehicle_file_price_prediction_df['Price_New_Final_Filled'].isna().sum()

In [ ]:
vehicle_file_price_prediction_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\grouped_price_data_new_price_filled.csv")

- Original missing

In [ ]:
print(vehicle_price_original_df['Price_New'].isna().sum())

print(vehicle_price_original_df['Price_Used_Average'].isna().sum())

In [ ]:
vehicle_file_price_prediction_df['Price_New_Final_Filled'].isna().sum()

In [ ]:
vehicle_file_price_prediction_df['Price_Used_Final_Filled'].isna().sum()


- Calculate depreciation/appreciation

In [ ]:
vehicle_file_price_prediction_df = vehicle_file_price_prediction_df.drop(columns=['Price_New_Median', 'Price_Used_Median', 'Price_New_Estimate_Filled', 'Price_Used_Mean_Estimate'])

In [ ]:
vehicle_file_price_prediction_df['End_Year_ABI_Code'] = vehicle_file_price_prediction_df['End_Date_ABI_Code'].apply(lambda x: (x.split('-'))[0] if ((x.split('-'))[0])!='9999' else -1)
vehicle_file_price_prediction_df['End_Year_ABI_Code'] = vehicle_file_price_prediction_df['End_Year_ABI_Code'].astype(float)

vehicle_file_price_prediction_df['Start_Year_ABI_Code'] = vehicle_file_price_prediction_df['Start_Date_ABI_Code'].apply(lambda x: (x.split('-'))[0] if ((x.split('-'))[0])!='9999' else -1)
vehicle_file_price_prediction_df['Start_Year_ABI_Code'] = vehicle_file_price_prediction_df['Start_Year_ABI_Code'].astype(float)

- Calculate decimal years from start of manufacturing

In [ ]:
import datetime

In [ ]:
vehicle_file_price_prediction_df['Years_From_First_Manufacturer']  = vehicle_file_price_prediction_df['Start_Date_ABI_Code'].apply(lambda x: round(((datetime.datetime.today() - datetime.datetime.strptime(x, '%Y-%m-%d')).days)/365, 2))


In [ ]:
vehicle_file_price_prediction_df['Years_From_First_Manufacturer'].max()

In [ ]:
vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['Price_New_Final_Filled']==0, 'Price_New_Final_Filled'] = np.full(len(vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['Price_New_Final_Filled']==0]), np.nan)

In [ ]:
vehicle_file_price_prediction_df['Price_Change_Metric'] = vehicle_file_price_prediction_df.apply(lambda x: ((((abs(x['Price_New_Final_Filled']-x['Price_Used_Final_Filled']))/x['Price_New_Final_Filled'])**(1/x['Years_From_First_Manufacturer']))) if (pd.isnull(x['Price_Used_Final_Filled'])==False and pd.isnull(x['Price_New_Final_Filled'])==False) else np.nan, axis = 1)

In [ ]:
vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['Price_Used_Final_Filled']>vehicle_file_price_prediction_df['Price_New_Final_Filled']][['ABI_Code_44', 'Manufacturer',
                                                                                                    'Model',
                                                                                                    'Veh_Generation',
                                                                                                    'Start_Year_Veh_Generation',
                                                                                                    'Veh_Desc_ABI',
                                                                                                    'Fuel_Primary',
                                                                                                    'Fuel_Secondary',
                                                                                                    'Engine_Size_Litres',
                                                                                                    'Body_Type',
                                                                                                    'Aspiration', 'Price_New_Final_Filled', 'Price_Used_Final_Filled']]

In [ ]:
vehicle_file_price_prediction_df.loc[(vehicle_file_price_prediction_df['Price_Used_Final_Filled']>vehicle_file_price_prediction_df['Price_New_Final_Filled'])&(vehicle_file_price_prediction_df['Manufacturer']=='Audi')&(vehicle_file_price_prediction_df['Model']=='Q2'), 'Price_Used_Final_Filled'] = np.full(len(vehicle_file_price_prediction_df.loc[(vehicle_file_price_prediction_df['Price_Used_Final_Filled']>vehicle_file_price_prediction_df['Price_New_Final_Filled'])&(vehicle_file_price_prediction_df['Manufacturer']=='Audi')&(vehicle_file_price_prediction_df['Model']=='Q2')]), 16500)

vehicle_file_price_prediction_df.loc[(vehicle_file_price_prediction_df['Price_Used_Final_Filled']>vehicle_file_price_prediction_df['Price_New_Final_Filled'])&(vehicle_file_price_prediction_df['Manufacturer']=='Audi')&(vehicle_file_price_prediction_df['Model']=='A3'), 'Price_Used_Final_Filled'] = np.full(len(vehicle_file_price_prediction_df.loc[(vehicle_file_price_prediction_df['Price_Used_Final_Filled']>vehicle_file_price_prediction_df['Price_New_Final_Filled'])&(vehicle_file_price_prediction_df['Manufacturer']=='Audi')&(vehicle_file_price_prediction_df['Model']=='A3')]), 49000)


In [ ]:
vehicle_file_price_prediction_df.loc[(vehicle_file_price_prediction_df['Price_Used_Final_Filled']>vehicle_file_price_prediction_df['Price_New_Final_Filled'])&(vehicle_file_price_prediction_df['Manufacturer']=='Audi')]['Start_Year_Veh_Generation']

- Manual Changes before model training

In [ ]:
vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==4137689, 'Price_Used_Final_Filled'] = 16500

vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==4137164, 'Price_Used_Final_Filled'] = 28500

vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==4136716, 'Price_Used_Final_Filled'] = 26000

vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==57100011, 'Price_Used_Final_Filled'] = 58000

vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==57100011, 'Price_New_Final_Filled'] = 69000

vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==57100011, 'Price_Used_Final_Filled'] = 58000




**Final_Price** = `Price_New_Final_Filled` - ((`Price_Change_Metric`^(`Years_From_First_Manufacturer`))*`Price_New_Final_Filled`)

In [ ]:
vehicle_file_price_prediction_df['Price_Change_Metric'].median()

In [ ]:
vehicle_file_price_prediction_df['Price_Change_Metric'].describe()

- Should I remove extremes...

- Use linear predictor/glm to calculate these missing values

In [ ]:
fuel_primary_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Fuel_Primary')
fuel_primary_map = dict(zip(fuel_primary_df['Key'], fuel_primary_df['Value']))

fuel_secondary_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Fuel_Secondary')
fuel_secondary_map = dict(zip(fuel_secondary_df['Key'], fuel_secondary_df['Value']))
fuel_secondary_map = {np.nan: 0, 'Electric': 1, 'CNG':2, 'Hydrogen': 3, 'LPG': 4, 'Petrol': 5, 'Biofuel': 6}

fuel_desc_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Fuel_Desc')
fuel_desc_map = dict(zip(fuel_desc_df['Key'], fuel_desc_df['Value']))

aspiration_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Aspiration')
aspiration_map = dict(zip(aspiration_df['Key'], aspiration_df['Value']))

drive_type_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Drive_Type')
drive_type_map = dict(zip(drive_type_df['Key'], drive_type_df['Value']))

body_type_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Body_Type')
body_type_map = dict(zip(body_type_df['Key'], body_type_df['Value']))

import_type_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Import_Type')
import_type_map = dict(zip(import_type_df['Key'], import_type_df['Value']))

gr20_l_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='GR20_L')
gr20_l_map = dict(zip(gr20_l_df['Key'], gr20_l_df['Value']))

gr50_l_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='GR50_L')
gr50_l_map = dict(zip(gr50_l_df['Key'], gr50_l_df['Value']))

transmission_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Transmission')
transmission_map = dict(zip(transmission_df['Key'], transmission_df['Value']))

transmission_group_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Transmission_Group')
transmission_group_map = dict(zip(transmission_group_df['Key'], transmission_group_df['Value']))

aeb_type_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='AEB_Type')
aeb_type_map = dict(zip(aeb_type_df['Key'], aeb_type_df['Value']))

speed_adapt_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Speed_Adapt')
speed_adapt_map = dict(zip(speed_adapt_df['Key'], speed_adapt_df['Value']))

speed_alert_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='Speed_Alert')
speed_alert_map = dict(zip(speed_alert_df['Key'], speed_alert_df['Value']))

ldw_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='LDW')
ldw_map = dict(zip(ldw_df['Key'], ldw_df['Value']))

blis_df = pd.read_excel(r"C:\Users\benwa\Downloads\Test_schema.xlsm", sheet_name='BLIS')
blis_map = dict(zip(blis_df['Key'], blis_df['Value']))

manufacturer_df = pd.read_excel(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\Categorical feature ordering-automated adjusted.xlsx", sheet_name='Manufacturers_Manual')
manufacturer_map = dict(zip(manufacturer_df['Manufacturer'], manufacturer_df['Rank']))

security_thatcham_map = {np.nan : 0, 'N2' : 1, 'N1' : 2, 'T2' : 3, 'T1' : 4}

In [ ]:
vehicle_file_price_prediction_numerical_df = vehicle_file_price_prediction_df.copy()

vehicle_file_price_prediction_numerical_df['Manufacturer'] = vehicle_file_price_prediction_numerical_df['Manufacturer'].apply(lambda x: manufacturer_map[x])

vehicle_file_price_prediction_numerical_df['Fuel_Primary'] = vehicle_file_price_prediction_numerical_df['Fuel_Primary'].apply(lambda x: fuel_primary_map[x])

vehicle_file_price_prediction_numerical_df['Fuel_Secondary'] = vehicle_file_price_prediction_numerical_df['Fuel_Secondary'].apply(lambda x: fuel_secondary_map[x])

vehicle_file_price_prediction_numerical_df['Fuel_Desc'] = vehicle_file_price_prediction_numerical_df['Fuel_Desc'].apply(lambda x: fuel_desc_map[x])

vehicle_file_price_prediction_numerical_df['Aspiration'] = vehicle_file_price_prediction_numerical_df['Aspiration'].apply(lambda x: aspiration_map[x])

vehicle_file_price_prediction_numerical_df['Drive_Type'] = vehicle_file_price_prediction_numerical_df['Drive_Type'].apply(lambda x: drive_type_map[x])

vehicle_file_price_prediction_numerical_df['Body_Type'] = vehicle_file_price_prediction_numerical_df['Body_Type'].apply(lambda x: body_type_map[x])

vehicle_file_price_prediction_numerical_df['Veh_Import_Type'] = vehicle_file_price_prediction_numerical_df['Veh_Import_Type'].apply(lambda x: import_type_map[x])

vehicle_file_price_prediction_numerical_df['GR20_L'] = vehicle_file_price_prediction_numerical_df['GR20_L'].apply(lambda x: gr20_l_map[x])

vehicle_file_price_prediction_numerical_df['GR50_L'] = vehicle_file_price_prediction_numerical_df['GR50_L'].apply(lambda x: gr50_l_map[x])

vehicle_file_price_prediction_numerical_df['Transmission'] = vehicle_file_price_prediction_numerical_df['Transmission'].apply(lambda x: transmission_map[x])

vehicle_file_price_prediction_numerical_df['Transmission_Group'] = vehicle_file_price_prediction_numerical_df['Transmission_Group'].apply(lambda x: transmission_group_map[x])

vehicle_file_price_prediction_numerical_df['AEB_Type'] = vehicle_file_price_prediction_numerical_df['AEB_Type'].apply(lambda x: aeb_type_map[x])

vehicle_file_price_prediction_numerical_df['Speed_Adapt'] = vehicle_file_price_prediction_numerical_df['Speed_Adapt'].apply(lambda x: speed_adapt_map[x])

vehicle_file_price_prediction_numerical_df['Speed_Alert'] = vehicle_file_price_prediction_numerical_df['Speed_Alert'].apply(lambda x: speed_alert_map[x])

vehicle_file_price_prediction_numerical_df['LDW'] = vehicle_file_price_prediction_numerical_df['LDW'].apply(lambda x: ldw_map[x])

vehicle_file_price_prediction_numerical_df['BLIS'] = vehicle_file_price_prediction_numerical_df['BLIS'].apply(lambda x: blis_map[x])

vehicle_file_price_prediction_numerical_df['Security_Thatcham'] = vehicle_file_price_prediction_numerical_df['Security_Thatcham'].apply(lambda x: security_thatcham_map[x])


In [ ]:
vehicle_file_price_prediction_numerical_df = vehicle_file_price_prediction_numerical_df.select_dtypes(np.number)

- Replace zeros with NaN

In [ ]:
vehicle_file_price_prediction_numerical_df.loc[vehicle_file_price_prediction_numerical_df['Price_New_Final_Filled']==0, 'Price_New_Final_Filled'] = np.full(len(vehicle_file_price_prediction_numerical_df.loc[vehicle_file_price_prediction_numerical_df['Price_New_Final_Filled']==0]), np.nan)

vehicle_file_price_prediction_numerical_df.loc[vehicle_file_price_prediction_numerical_df['Price_Used_Final_Filled']==0, 'Price_Used_Final_Filled'] = np.full(len(vehicle_file_price_prediction_numerical_df.loc[vehicle_file_price_prediction_numerical_df['Price_Used_Final_Filled']==0]), np.nan)


- Is there new price data

In [ ]:
print(len(vehicle_file_price_prediction_numerical_df))

print(len(vehicle_file_price_prediction_numerical_df.loc[pd.isnull(vehicle_file_price_prediction_numerical_df['Price_New_Final_Filled'])==False]))


In [ ]:
Vehicle_file_predictions_all = vehicle_file_price_prediction_numerical_df.loc[pd.isnull(vehicle_file_price_prediction_numerical_df['Price_New_Final_Filled'])==False]

Vehicle_file_predictions_all = Vehicle_file_predictions_all[X.columns]

In [ ]:
final_predictions = vehicle_file_price_prediction_numerical_df.loc[pd.isnull(vehicle_file_price_prediction_numerical_df['Price_Change_Metric'])==False]
final_predictions

In [ ]:
vehicle_file_price_prediction_numerical_df.loc[(pd.isnull(vehicle_file_price_prediction_numerical_df['Price_New_Final_Filled'])==True)&(pd.isnull(vehicle_file_price_prediction_numerical_df['Price_Used_Final_Filled'])==False)]

# Imputing missing values

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
vehicle_file_price_prediction_numerical_copy_df = vehicle_file_price_prediction_numerical_df.copy()

vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['GR_L']==-1, 'GR_L'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['GR_L']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['ID_Score_NVSA']==-1, 'ID_Score_NVSA'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['ID_Score_NVSA']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Body_Score_NVSA']==-1, 'Body_Score_NVSA'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Body_Score_NVSA']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Electronic_Score_NVSA']==-1, 'Electronic_Score_NVSA'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Electronic_Score_NVSA']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Safety_Manufacturer']==-1, 'Safety_Manufacturer'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Safety_Manufacturer']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['AEB_Type']==-1, 'AEB_Type'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['AEB_Type']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['AEB_Score']==-1, 'AEB_Score'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['AEB_Score']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Speed_Alert']==-1, 'Speed_Alert'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Speed_Alert']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Speed_Adapt']==-1, 'Speed_Adapt'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Speed_Adapt']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Bumper_Score_RCAR']==-1, 'Bumper_Score_RCAR'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Bumper_Score_RCAR']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['LDW']==-1, 'LDW'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['LDW']==-1]), np.nan)
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['BLIS']==-1, 'BLIS'] = np.full(len(vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['BLIS']==-1]), np.nan)

vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['ID_Score_NVSA']==0, 'ID_Score_NVSA'] = np.nan
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['ID_Score_NVSA']==1, 'ID_Score_NVSA'] = np.nan

vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Body_Score_NVSA']==0, 'Body_Score_NVSA'] = np.nan
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Body_Score_NVSA']==1, 'Body_Score_NVSA'] = np.nan

vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Electronic_Score_NVSA']==0, 'Electronic_Score_NVSA'] = np.nan
vehicle_file_price_prediction_numerical_copy_df.loc[vehicle_file_price_prediction_numerical_copy_df['Electronic_Score_NVSA']==1, 'Electronic_Score_NVSA'] = np.nan

imputer = IterativeImputer(max_iter=10, random_state=0)

imputed_values = imputer.fit_transform(vehicle_file_price_prediction_numerical_copy_df)

In [ ]:
imputed_values.shape

In [ ]:
vehicle_file_price_prediction_numerical_copy_df[vehicle_file_price_prediction_numerical_copy_df.columns] = imputed_values

vehicle_file_price_prediction_numerical_copy_df

In [ ]:
for val in list(vehicle_file_price_prediction_numerical_df.columns):
    print(val)
    print(len(vehicle_file_price_prediction_numerical_df.loc[vehicle_file_price_prediction_numerical_df[val]==-1]))
    
    print(len(vehicle_file_price_prediction_numerical_df.loc[pd.isnull(vehicle_file_price_prediction_numerical_df[val])==True]))
    

# Overlay

In [ ]:
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)

In [ ]:
vehicle_file_price_prediction_numerical_df

In [ ]:
vehicle_file_price_prediction_numerical_df['GR_L'] = vehicle_file_price_prediction_numerical_df.apply(lambda x: x['GR50_L'] if x['GR50_L'] !=-1 else x['GR20_L'], axis = 1)

vehicle_file_price_prediction_numerical_df['GR_L'].unique()

In [ ]:
vehicle_file_price_prediction_numerical_df.loc[vehicle_file_price_prediction_numerical_df['GR_L']==-1]

In [ ]:
vehicle_file_price_prediction_numerical_df['GR_L'].unique()

- Imputing final missing prices

In [ ]:
scaler = MinMaxScaler()

df_overlay = vehicle_file_price_prediction_numerical_copy_df[['Years_From_First_Manufacturer', 'vehicle_features_MR_Risk_Accident_23', 'GR_L', 'Safety_Manufacturer', 'Power_to_Weight', 'Fuel_Desc', 'Veh_Import_Type', 'Body_Type', 'Seats_Num', 'Doors_Num', 'Volume_Box', 'Engine_Size_PetroElectric', 'ID_Score_NVSA', 'Body_Score_NVSA', 'Electronic_Score_NVSA', 'Engine_Power_kW']]

df_overlay_scaled = scaler.fit_transform(df_overlay.to_numpy())

df_overlay.loc[:, df_overlay.columns] = df_overlay_scaled

df_overlay = vehicle_file_price_prediction_numerical_copy_df[['ABI_Code_44']].join(df_overlay)

df_overlay

In [ ]:
body_type_map

In [ ]:
df_overlay


In [ ]:
import_type_map

- Incorporating Price data

In [ ]:
price_df = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\vehicle_file_with_depreciation_factor.csv")

In [ ]:
print(len(price_df))
print(len(price_df.loc[pd.isnull(price_df['Price_New_Final_Filled'])==False]))

In [ ]:
price_df

In [ ]:
price_combined_df = price_df.copy()

price_combined_df['Price_Used_Final_Filled'] = price_combined_df.apply(lambda x: x['Price_Used_Final_Filled'] if pd.isnull(x['Price_Used_Final_Filled'])==False else x['Price_Used_Predicted'] if x['Price_Used_Predicted']>300 else np.nan, axis = 1)

In [ ]:
price_combined_df['Price_New_Final_Filled'].isna().sum()


In [ ]:
price_combined_df['Price_Used_Final_Filled'].isna().sum()

In [ ]:
imputer = IterativeImputer(max_iter=10, random_state=0)

price_combined_copy_df = price_combined_df.copy()

price_combined_copy_df = price_combined_copy_df.select_dtypes(include='number')

imputed_values = imputer.fit_transform(price_combined_copy_df)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler((-3,3))

In [ ]:
df_overlay['Overlay_Value'] = - (1.5*df_overlay['Electronic_Score_NVSA']) - (1.2*df_overlay['GR_L']) - (0.8*df_overlay['Body_Score_NVSA']) - (0.6*df_overlay['ID_Score_NVSA']) + df_overlay['vehicle_features_MR_Risk_Accident_23'] + df_overlay['Veh_Import_Type'] #  + df_overlay['Power_to_Weight'] + df_overlay['Safety_Manufacturer'] + df_overlay['Engine_Power_kW']

#+ (0.4*df_overlay['Safety_Manufacturer']) + (0.6*df_overlay['Engine_Power_kW']) + (0.8*df_overlay['Power_to_Weight']) + (0.8*df_overlay['vehicle_features_MR_Risk_Accident_23']) + (0.8*df_overlay['Veh_Import_Type']) + (0.6*df_overlay['Engine_Power_kW'])

#(0.4*df_overlay['Safety_Manufacturer'])+(0.3*df_overlay['Engine_Power_kW'])+ (0.6*df_overlay['Power_to_Weight']) + (0.4*df_overlay['vehicle_features_MR_Risk_Accident_23']) +(0.6*df_overlay['Veh_Import_Type']) + (0.4*df_overlay['Engine_Power_kW'])

a, b = -2, 2

x,y = df_overlay['Overlay_Value'].min(), df_overlay['Overlay_Value'].max()

#df_overlay['Overlay_Value_Scaled'] = df_overlay['Overlay_Value'].apply(lambda overlay: round((overlay-x)/(y-x)*(b-a)+a))

df_overlay['Overlay_Value_Scaled'] = scaler.fit_transform(df_overlay[['Overlay_Value']])

In [ ]:
plt.plot(df_overlay['Overlay_Value_Scaled'])

In [ ]:
df_overlay.loc[df_overlay['Overlay_Value_Scaled']<-1]


In [ ]:
master_file_with_overlay_df = master_file.join(df_overlay['Overlay_Value_Scaled'])

In [ ]:
master_file_with_overlay_df['Overlay_Value_Scaled'] = df_overlay['Overlay_Value_Scaled']

In [ ]:
master_file_with_overlay_df['Overlay_Value_Scaled'].unique()

In [ ]:
master_file_with_overlay_df.loc[master_file_with_overlay_df['Overlay_Value_Scaled']==-2]

In [ ]:
master_file_with_overlay_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Overlays_try\possible_overlay.csv")

In [ ]:
grouped_by_manufacturer = master_file_with_overlay_df.groupby(by=['Manufacturer', 'Overlay_Value_Scaled']).agg({'Overlay_Value_Scaled':['count', 'first']}).reset_index()

labels = grouped_by_manufacturer['Overlay_Value_Scaled']['first']

def addlabels(x,y):
    for i in range(len(x)):
        plt.text(x[i], y[i], labels[i], ha = 'center')

In [ ]:
grouped_by_manufacturer = master_file_with_overlay_df.groupby(by=['Manufacturer']).agg({'Overlay_Value_Scaled':['mean', 'median']}).reset_index()


x = grouped_by_manufacturer['Manufacturer']

y = grouped_by_manufacturer['Overlay_Value_Scaled']['mean']


plt.figure(figsize=(20, 15))
plt.bar(x,y)
plt.xticks(rotation=90)

#addlabels(x, y)

plt.show()



In [ ]:
master_file_with_overlay_df

In [ ]:
df_overlay['Overlay_Value'].max()

In [ ]:
master_df_check = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\08_Master_File\vehicle_features_Vehicle_File_Master_05_09_2023.csv")

New and Used need to train data

In [ ]:
vehicle_file_price_prediction_numerical_new_price_present_df = vehicle_file_price_prediction_numerical_df.loc[(pd.isnull(vehicle_file_price_prediction_numerical_df['Price_New_Final_Filled'])==False)&(pd.isnull(vehicle_file_price_prediction_numerical_df['Price_Used_Final_Filled'])==False)]

In [ ]:
len(vehicle_file_price_prediction_numerical_new_price_present_df)

In [ ]:
vehicle_file_price_prediction_numerical_new_price_present_df.columns

In [ ]:
from sklearn.model_selection import train_test_split 

from sklearn import linear_model 
from sklearn.metrics import r2_score 

In [ ]:
X = vehicle_file_price_prediction_numerical_new_price_present_df.drop(columns=['Price_Used_Final_Filled', 'Data_Version', 'Price_Change_Metric'])
y = vehicle_file_price_prediction_numerical_new_price_present_df['Price_Change_Metric']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
len(vehicle_file_price_prediction_numerical_new_price_present_df)

- Linear Model sklearn

In [ ]:
lm = linear_model.LinearRegression() 
lm.fit(X_train, y_train) 
y_pred = lm.predict(X_test) 
print(r2_score(y_true=y_test, y_pred=y_pred)) # 0.7518 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
importance = lm.coef_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature:', list(vehicle_file_price_prediction_numerical_new_price_present_df.columns)[i], 'Score: %.5f' % (v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

- OLS Stats models

In [ ]:
import statsmodels.api as sm 

In [ ]:
model_ols = sm.OLS(y_train, X_train).fit() 
predictions = model_ols.predict(X_test) 
print(model_ols.rsquared) # 0.91823 

- CatBoost regressor

In [ ]:
from catboost import CatBoostRegressor 

In [ ]:
model_cat_boost = CatBoostRegressor(iterations=6542, learning_rate=0.1) 
model_cat_boost.fit( 
	X_train, y_train, 
	eval_set=(X_test, y_test), 
) 
print(model_cat_boost.score(X, y)) # 0.92416 

In [ ]:
model_cat_boost.predict(X_test)

In [ ]:
print(model_cat_boost.score(X, y))

In [ ]:
importances = model_cat_boost.get_feature_importance(type='PredictionValuesChange')
feature_importances = pd.Series(importances, index=X.columns).sort_values()

In [ ]:
plt.figure(figsize=(10, 6))

plt.barh(feature_importances.index, feature_importances.values)

- Save model

In [ ]:
model_cat_boost.save_model(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\catBoost_price_depreciation_model.cbm")

In [ ]:
model_catboost_loaded = CatBoostRegressor(
    random_seed=42,
    logging_level='Silent')
model_catboost_loaded.load_model(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\catBoost_price_depreciation_model.cbm")

- Making predictions on all data

In [ ]:
model_predicted = model_cat_boost.predict(Vehicle_file_predictions_all)

In [ ]:
predictions_dict = dict(zip(Vehicle_file_predictions_all['ABI_Code_44'], model_predicted))
predictions_dict

In [ ]:
vehicle_file_price_prediction_df['Price_Change_Metric_Model'] = vehicle_file_price_prediction_df['ABI_Code_44'].apply(lambda x: predictions_dict[x] if x in list(predictions_dict.keys()) else np.nan)

In [ ]:
vehicle_file_price_prediction_df['Price_Change_Metric_Model'].isna().sum()

- Ensuring no Price Changes calculated when price data is missing

In [ ]:
vehicle_file_price_prediction_df.loc[(pd.isnull(vehicle_file_price_prediction_df['Price_New_Final_Filled'])==True)|(pd.isnull(vehicle_file_price_prediction_df['Price_Used_Final_Filled'])==True), 'Price_Change_Metric'] = np.full(len(vehicle_file_price_prediction_df.loc[(pd.isnull(vehicle_file_price_prediction_df['Price_New_Final_Filled'])==True)|(pd.isnull(vehicle_file_price_prediction_df['Price_Used_Final_Filled'])==True)]), np.nan)

In [ ]:
vehicle_file_price_prediction_df.loc[(pd.isnull(vehicle_file_price_prediction_df['Price_Change_Metric'])==False)&(pd.isnull(vehicle_file_price_prediction_df['Price_Change_Metric_Model'])==False)]

- Calculating predicted used price

**Final_Price** = `Price_New_Final_Filled` - ((`Price_Change_Metric`^(`Years_From_First_Manufacturer`))*`Price_New_Final_Filled`)

In [ ]:
vehicle_file_price_prediction_df['Price_Used_Predicted'] = vehicle_file_price_prediction_df['Price_New_Final_Filled'] - (((vehicle_file_price_prediction_df['Price_Change_Metric_Model'])**(vehicle_file_price_prediction_df['Years_From_First_Manufacturer']))*vehicle_file_price_prediction_df['Price_New_Final_Filled'])

In [ ]:
vehicle_file_price_prediction_df.loc[pd.isnull(vehicle_file_price_prediction_df['Price_Used_Final_Filled'])==True]

In [ ]:
vehicle_file_price_prediction_df.loc[abs(vehicle_file_price_prediction_df['Price_Used_Predicted']-vehicle_file_price_prediction_df['Price_Used_Final_Filled'])>1000]

In [ ]:
plt.scatter(vehicle_file_price_prediction_df['Price_Used_Final_Filled'], vehicle_file_price_prediction_df['Price_Used_Predicted'])

plt.xlim(0,100000)

plt.ylim(0,100000)


- Pretty good predictions post 2010-falls off after price drops too low


In [ ]:
vehicle_file_price_prediction_df.loc[(pd.isnull(vehicle_file_price_prediction_df['Price_Used_Final_Filled'])==True)&(vehicle_file_price_prediction_df['Price_Used_Predicted']<500)]#['Price_Used_Predicted'].min()

In [ ]:
vehicle_file_price_prediction_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\vehicle_file_with_depreciation_factor.csv")

In [ ]:
vehicle_file_price_prediction_df.loc[pd.isnull(vehicle_file_price_prediction_df['Price_New_Final_Filled'])==True]

In [ ]:
vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==11700003, 'Price_New_Final_Filled'] = 20000

vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==11700003, 'Price_Used_Final_Filled'] = 12000


- Using model and engine information...

In [ ]:
dummy_model_df = pd.get_dummies(vehicle_file_price_prediction_df['Model'], dtype=int)

#dummy_trim_df = pd.get_dummies(vehicle_file_price_prediction_df['Trim'], dtype=int)

dummy_engine_label_df = pd.get_dummies(vehicle_file_price_prediction_df['Engine_Label'], dtype=int)


In [ ]:
model_only_df = dummy_model_df

model_and_engine_numeric_df = dummy_model_df.join(dummy_engine_label_df, lsuffix='_model', rsuffix='_engine')

#model_and_engine_and_trim_numeric_df = model_and_engine_numeric_df.join(dummy_trim_df, lsuffix='_model', rsuffix='_engine')

In [ ]:
vehicle_file_price_prediction_model_engine_encoded_df = vehicle_file_price_prediction_df.join(model_and_engine_numeric_df)

In [ ]:
vehicle_file_price_prediction_model_engine_encoded_df.loc[vehicle_file_price_prediction_model_engine_encoded_df['Price_New_Estimate']==0, 'Price_New_Estimate'] = np.full(len(vehicle_file_price_prediction_model_engine_encoded_df.loc[vehicle_file_price_prediction_model_engine_encoded_df['Price_New_Estimate']==0]), np.nan)

vehicle_file_price_prediction_model_engine_encoded_df.loc[vehicle_file_price_prediction_model_engine_encoded_df['Price_Used_Average_Estimate']==0, 'Price_Used_Average_Estimate'] = np.full(len(vehicle_file_price_prediction_model_engine_encoded_df.loc[vehicle_file_price_prediction_model_engine_encoded_df['Price_Used_Average_Estimate']==0]), np.nan)

vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df = vehicle_file_price_prediction_model_engine_encoded_df.loc[(pd.isnull(vehicle_file_price_prediction_model_engine_encoded_df['Price_New_Estimate'])==False)&(pd.isnull(vehicle_file_price_prediction_model_engine_encoded_df['Price_Used_Average_Estimate'])==False)]

vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df = vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df.select_dtypes(np.number)

X_model_engine = vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df.drop(columns=['Price_Used_Average_Estimate'], axis=1)
y_model_engine = vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df['Price_Used_Average_Estimate']


In [ ]:
len(y_model_engine.loc[y_model_engine<500])

- Create trainign and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_model_engine, y_model_engine, train_size=0.7, test_size=0.3, random_state=100)


In [ ]:
model = CatBoostRegressor(iterations=6542, learning_rate=0.05) 
model.fit( 
	X_train, y_train, 
	eval_set=(X_test, y_test), 
) 
print(model.score(X_model_engine, y_model_engine)) # 0.92416 

- OLS Stats models

In [ ]:
import statsmodels.api as sm 

In [ ]:
model = sm.OLS(y_train, X_train).fit() 
predictions = model.predict(X_test) 
print(model.rsquared) 

In [ ]:
y_train.min()

In [ ]:
X_test

In [ ]:
vehicle_file_price_prediction_df['Price_Change'] = vehicle_file_price_prediction_df['Price_New_Estimate']/vehicle_file_price_prediction_df['Price_Used_Average_Estimate']

vehicle_file_price_prediction_df.loc[(vehicle_file_price_prediction_df['Price_Change']<1)][['Manufacturer', 'Model', 'Body_Type', 'Veh_Desc_ABI', 'Start_Year_Veh_Generation', 'Price_New_Estimate', 'Price_Used_Average_Estimate']]

In [ ]:
vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['Price_Change']<1]

In [ ]:
vehicle_file_price_prediction_df['Price_Change'].describe()

- Joining onto main file

In [ ]:
final_predictions = vehicle_file_price_prediction_numerical_df.loc[pd.isnull(vehicle_file_price_prediction_numerical_df['Price_New_Estimate'])==False]
final_predictions = final_predictions.drop(columns = ['Price_Used_Average_Estimate'])

final_predictions = final_predictions.join(model_and_engine_numeric_df)

final_predictions = final_predictions.select_dtypes(np.number)

final_predictions.isna().sum().sum()

In [ ]:
final_predictions = final_predictions[list(X_test.columns)]

In [ ]:
final_predictions.shape

In [ ]:
predict_all = model.predict(final_predictions)
len(predict_all)

In [ ]:
final_predictions_dict = dict(zip(final_predictions['ABI_Code_44'], predict_all))

In [ ]:
final_predictions_dict.keys()

In [ ]:
vehicle_file_price_prediction_df['Price_Used_Model'] = vehicle_file_price_prediction_df['ABI_Code_44'].apply(lambda x: final_predictions_dict[x] if x in (final_predictions_dict.keys()) else np.nan)

In [ ]:
vehicle_file_price_prediction_df['Price_Used_Model'].isna().sum()

In [ ]:
vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['Price_Used_Model']<500]

In [ ]:
vehicle_file_price_prediction_df.to_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\16_Theft_Rating_Re_Worked\Price_Analysis\vehicle_file_with_used_price_prediction.csv")

In [ ]:
vehicle_file_price_prediction_df

In [ ]:
vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df['Price_Used_Model'] = predict_all

In [ ]:
vehicle_file_with_prediction_df = vehicle_file_price_prediction_numerical_df.merge(vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df[['ABI_Code_44', 'Price_Used_Model']], on = ['ABI_Code_44', 'ABI_Code_44'], how='left')

In [ ]:
vehicle_file_price_prediction_numerical_df['ABI_Code_44']

In [ ]:
vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df['ABI_Code_44']

In [ ]:
vehicle_file_with_prediction_df['Price_Used_Model'].isna().sum()

In [ ]:
vehicle_file_price_prediction_numerical_model_engine_encoded_new_price_present_df['Price_Used_Model'].isna().sum()

- Plots!!!

In [ ]:
plt.scatter(predictions, y_test)

plt.xlim(-1000,150000)
plt.ylim(-1000,150000)


- Using model, engine_label and trim

- Using model and engine_label

Score: 0.9668099721490522

In [ ]:
plt.plot(y_test - predictions, marker='o', linestyle='')

plt.ylim(-20000,20000)


- Not using model or engine_label encoded

In [ ]:
plt.plot(y_test - predictions, marker='o', linestyle='')

plt.ylim(-20000,20000)


In [ ]:
X_test.loc[887]

In [ ]:
vehicle_file_price_prediction_df.loc[vehicle_file_price_prediction_df['ABI_Code_44']==3511563]

0.8806879694963347

- Filling in missing values

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
columns_list = list(X_Copy.columns)
columns_list.remove('Bumper_Score_RCAR')

In [ ]:
for col_header in columns_list:
    X_Copy.loc[X_Copy[col_header]==-1, col_header] = np.full(len(X_Copy.loc[X_Copy[col_header]==-1]), np.nan)
X_Copy

In [ ]:
#X_Copy = X_Copy.drop(columns = ['Security_Thatcham', 'Fuel_Desc'])
imputer = KNNImputer(n_neighbors=3)
imputed_data = imputer.fit_transform(X_Copy)

In [ ]:
imputed_df = pd.DataFrame(imputed_data, columns=X_Copy.columns)

imputed_df

In [ ]:
X_Copy

In [ ]:
X_Copy['Security_Thatcham_Numeric']*X_Copy['Volume_Box']

In [ ]:
X_Copy['Theft_Uplift'] = 

In [ ]:
import plotly.express as px

In [ ]:
px.bar()

#### Provisional theft data

In [ ]:
# Groups:
# 0 - theft decreases in likelihood with age (joy ride class)
# 1 - theft increases in likelihood with age (scrap class)
# 2 - theft increases in likelihood around mean (rare class)
manufacturer_band = {
 'ABARTH': 1,
 'AC': 2,
 'AIXAM': 0,
 'ALFA ROMEO': 1,
 'ALPINA': 2,
 'ALPINE': 2,
 'ASIA': 0,
 'ASTON MARTIN': 2,
 'AUDI': 1,
 'BENTLEY': 2,
 'BMW': 1,
 'BRISTOL': 2,
 'BUGATTI': 2,
 'BUICK': 1,
 'BYD': 0,
 'CADILLAC': 1,
 'CATERHAM': 0,
 'CHEVROLET': 0,
 'CHRYSLER': 1,
 'CITROEN': 0,
 'CUPRA': 1,
 'DACIA': 0,
 'DAEWOO': 0,
 'DAIHATSU': 0,
 'DAIMLER': 2,
 'DE TOMASO': 2,
 'DFSK': 0,
 'DODGE': 2,
 'DS': 1,
 'FERRARI': 2,
 'FIAT': 0,
 'FORD': 0,
 'GARDNER DOUGLAS': 2,
 'GEM': 1,
 'GENESIS': 2,
 'GINETTA': 2,
 'HINDUSTAN': 0,
 'HOLDEN': 0,
 'HONDA': 0,
 'HUMMER': 2,
 'HYUNDAI': 0,
 'INEOS': 2,
 'INFINITI': 2,
 'ISUZU': 0,
 'JAGUAR': 1,
 'JEEP': 1,
 'KIA': 0,
 'KOENIGSEGG': 2,
 'LADA': 0,
 'LAMBORGHINI': 2,
 'LANCIA': 2,
 'LAND ROVER': 2,
 'LEXUS': 1,
 'LIGIER': 2,
 'LINCOLN': 1,
 'LOTUS': 2,
 'MARANELLO': 0,
 'MARCOS': 2,
 'MARLIN': 2,
 'MASERATI': 2,
 'MAXUS': 1,
 'MAYBACH': 2,
 'MAZDA': 0,
 'MCLAREN': 2,
 'MERCEDES': 1,
 'MEV': 0,
 'MG': 0,
 'MG MOTOR UK': 0,
 'MINI': 1,
 'MITSUBISHI': 0,
 'MITSUOKA': 2,
 'MORGAN': 2,
 'MOSLER': 2,
 'MUNRO': 2,
 'MW MOTORS S.R.O.': 2,
 'MYCAR': 0,
 'NAC MG': 0,
 'NISSAN': 0,
 'NOBLE': 2,
 'OKA': 0,
 'OPEL': 0,
 'PANTHER': 1,
 'PERODUA': 0,
 'PEUGEOT': 0,
 'PGO': 2,
 'POLESTAR': 1,
 'PONTIAC': 1,
 'PORSCHE': 2,
 'PROTON': 0,
 'RELIANT': 0,
 'RENAULT': 0,
 'ROLLS': 2,
 'ROVER': 0,
 'SAAB': 0,
 'SAN': 0,
 'SAO': 0,
 'SEAT': 0,
 'SHANGHIEDRIVE': 1,
 'SKODA': 0,
 'SMART': 1,
 'SSANGYONG': 0,
 'STEVENS': 0,
 'SUBARU': 0,
 'SUZUKI': 0,
 'T.V.R.': 2,
 'TATA': 0,
 'TD': 0,
 'TESLA': 1,
 'THINK': 1,
 'TORNADO': 0,
 'TOYOTA': 0,
 'VAUXHALL': 0,
 'VENTURI': 2,
 'VEXEL': 0,
 'VOLKSWAGEN': 0,
 'VOLVO': 1,
 'WESTFIELD': 2,
 'ZENOS': 2
        }

In [ ]:
len(set(licensed_vehicle_df["Make"]))

In [ ]:
# commonality
commonality = {}
boundaries = [0, 1, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000,
          20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 10000000]
for y in full_model_count.keys():
    for model in full_model_count[y].keys():
        if model not in commonality.keys():
            commonality[model] = []
        value = full_model_count[y][model]
        for i in range(len(boundaries)):
            if value >= boundaries[i] and value < boundaries[i+1]:
                commonality[model].append(i+1)
                
#for model in theft_values.keys():
for model in commonality.keys():
    commonality[model] = int(sum(commonality[model])/len(commonality[model]))

In [ ]:
commonality

In [ ]:
# make bonus
make_model_bonus = {}
for model in commonality.keys():
    matched_make = ""
    for make in make_model_year.keys():
        if model in make_model_year[make]:
            matched_make = make
    make_model_bonus[model] = 10*manufacturer_band[matched_make] if matched_make in manufacturer_band.keys() else 0

In [ ]:
commonality.keys()

In [ ]:
theft_values = {}
for model in make_model_bonus.keys():
    theft_values[model] = commonality[model]+make_model_bonus[model]

In [ ]:
date_shift = {0: [1990, 30, [0, -1]],
              1: [1993, 0, [1, 1]],
              2: [2013, 20, [1, 1]]}

In [ ]:
theft_values_df = pd.DataFrame(columns=["GenModel","Theft_rating"])
theft_values_df = theft_values_df.assign(GenModel=theft_values.keys(), Theft_rating=theft_values.values())

In [ ]:
theft_values_df

In [ ]:
theft_values_df.loc[theft_values_df['GenModel'] == 'KIA RIO']

In [ ]:
theft_values_df.loc[theft_values_df['GenModel'] == 'HYUNDAI ACCENT']

### Vehicle file

In [ ]:
data_file = pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\02_Vehicle_File_Collaboration\Public_Fills\vehicle_features_Vehicle_File_Best_Matched_With_Local_Fills_Ben_New.csv")
data_file["Manufacturer"] = data_file["Manufacturer"].replace({"MG Motor UK": "MGMUK"})
data_file["Manufacturer"] = data_file["Manufacturer"].apply(lambda x: x.upper().split(" ",1)[0].split("-",1)[0])
data_file["Manufacturer"] = data_file["Manufacturer"].replace({"LAND": "LAND ROVER", "ALFA": "ALFA ROMEO", "ASTON": "ASTON MARTIN",
                                                              "DE": "DE TOMASO", "GARDNER": "GARDNER DOUGLAS", "MW": "MW MOTORS S.R.O.", "NAC": "NAC MG",
                                                              "MGMUK": "MG MOTOR UK"})
data_file["Model"] = data_file["Model"].str.upper()
data_file["GenModel"] = data_file["Manufacturer"]+" "+data_file["Model"]

## necessary_factors
data_file = data_file.drop(columns={"Weight_Kerb", "Acceleration","Top_Speed"})
data_file = data_file.merge(pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\10_Columns_for_review\02_08_2023\Acceleration\Acceleration_TVI.csv"),on="TVI_Code",how="left")
data_file = data_file.merge(pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\10_Columns_for_review\02_08_2023\Top Speed\Top_Speed_TVI.csv"),on="TVI_Code",how="left")
data_file = data_file.merge(pd.read_csv(r"C:\Users\benwa\vehicle_features\Products - Documents\03_Vehicle_File\10_Columns_for_review\04_07_2023\Dimensions Review\Files for Match Replace\abi_dimensions_final_20230801.csv")[["TVI_Code", "Weight_Kerb_Fill", "Volume_Box"]].rename(columns={"Weight_Kerb_Fill": "Weight_Kerb"}),on="TVI_Code",how="left")

In [ ]:
data_file['GenModel'].unique()

### Factor production

#### Accident calculation

In [ ]:
# direct match
accident_matched_file = data_file.merge(accident_values_df, on='GenModel', how="left")

In [ ]:
accident_matched_file

In [ ]:
set(accident_matched_file["Accident_rating"].dropna())

In [ ]:
# bespoke value fill
luxury_makes = ["FERRARI", "BUGATTI", "LAMBORGHINI", "ASTON MARTIN", "T.V.R.", "ROLLS", "MCLAREN", "KOENIGSEGG", "DE TOMASO"]
for make in luxury_makes:
    accident_matched_file.loc[accident_matched_file['Manufacturer'] == make, "Accident_rating"] = 99

In [ ]:
accident_matched_file.loc[(accident_matched_file['Manufacturer']=='AUDI')&(accident_matched_file['Model']=='A3')]["Accident_rating"].unique()

In [ ]:
# indirect match
from difflib import SequenceMatcher

indirect_matches = {}
remaining_df = accident_matched_file[accident_matched_file["Accident_rating"].isna()]
remaining_models = list(set(remaining_df["GenModel"]))
print(len(remaining_models))
for i in range(len(remaining_models)):
    if i % 100 == 0:
        print(i)
    mx, match = 0.875, 0
    for j, row in accident_values_df.iterrows():
        veh = row["GenModel"]
        val = SequenceMatcher(None, veh, remaining_models[i]).ratio()
        if veh in remaining_models[i]:
            val = 1
        if val > mx:
            mx = val
            match = veh
    if match != 0:
        # print(remaining_models[i] + " - " + match)
        rarity_detriment = []
        indirect_matches[remaining_models[i]] = list(accident_values_df[accident_values_df["GenModel"]==match]["Accident_rating"])[0]
for k in indirect_matches.keys():
    accident_matched_file.loc[accident_matched_file['GenModel'] == k, "Accident_rating"] = indirect_matches[k]

In [ ]:
indirect_matches

In [ ]:
# ## Manufacturer vehicle imputation
remaining_models = list(set(accident_matched_file[accident_matched_file["Accident_rating"].isna()]["GenModel"]))
matched_manufacturers = list(set(accident_matched_file[accident_matched_file["Accident_rating"].notna()][accident_matched_file["Accident_rating"] < 99]["Manufacturer"]))
remaining_models_with_matched_make = []
for model in remaining_models:
    x = False
    for make in matched_manufacturers:
        x |= model.split()[0] in make.split()
    if x:
        remaining_models_with_matched_make.append(model)
print(len(remaining_models_with_matched_make))
distance_factors = ["Volume_Box","Weight_Kerb","Acceleration","Top_Speed"]
matched_ratings = {}
for model in remaining_models_with_matched_make:
    print(model)
    closest_dist, closest_point, closest_model = 100, [], ""
    model_make = ""
    for make in matched_manufacturers:
        make_split = make.split()
        model_split = model.split()
        x = True
        if len(make_split) < len(model_split):
            for i in range(len(make_split)):
                x &= make_split[i] == model_split[i]
            if x:
                model_make = make
    print(model_make)
    make_df = accident_matched_file[accident_matched_file["Manufacturer"]==model_make][distance_factors+["GenModel","Accident_rating"]]
    mean, std = [], []
    for factor in distance_factors:
        mean.append(make_df[factor].mean())
        std.append(make_df[factor].std())
    new_model_df = accident_matched_file[accident_matched_file["GenModel"]==model][distance_factors]
    for i in range(len(distance_factors)):
        new_model_df[distance_factors[i]] = (new_model_df[distance_factors[i]]-mean[i])/std[i]
    mean_point = []
    for f in distance_factors:
        mean_point.append(new_model_df[f].mean())
    # print(mean_point)
    matched_model_df = make_df[make_df["Accident_rating"].notna()]
    matched_model_list = list(set(matched_model_df["GenModel"]))
    # print("MATCHED MODEL")
    for matched_model in matched_model_list:
        current_matched_model_df = matched_model_df[matched_model_df["GenModel"]==matched_model]
        for i in range(len(distance_factors)):
            current_matched_model_df[distance_factors[i]] = (current_matched_model_df[distance_factors[i]]-mean[i])/std[i]
        matched_mean_point = []
        for f in distance_factors:
            matched_mean_point.append(current_matched_model_df[f].mean())
        dist = 0
        for i in range(len(mean_point)):
            if not math.isnan(mean_point[i]):
                dist += math.pow((mean_point[i]-matched_mean_point[i]),2)
        dist = math.sqrt(dist)
        if closest_dist > dist:
            closest_dist = dist
            closest_point = matched_mean_point
            closest_model = matched_model
    mx, commonality_match = 0.875, closest_model
    for j, row in accident_values_df.iterrows():
        veh = row["GenModel"]
        val = SequenceMatcher(None, veh, closest_model).ratio()
        if veh in closest_model:
            val = 1
        if val > mx:
            mx = val
            commonality_match = veh
    print(closest_model)
    print(commonality_match)
    # print(closest_point)
    # print(closest_dist)
    commonality_detriment = []
    for y in years:
        if commonality_match in commonality[y].keys():
            commonality_detriment.append(commonality[y][commonality_match])
    commonality_detriment = sum(commonality_detriment)/(len(commonality_detriment)*2)
    matched_ratings[model] = int(list(accident_matched_file[accident_matched_file["GenModel"]==closest_model]["Accident_rating"])[0]-commonality_detriment)
    print(matched_ratings[model])
    print()
print(matched_ratings)

In [ ]:
# Non-manufacturer vehicle imputation
remaining_models_without_matched_make = set(remaining_models)-set(remaining_models_with_matched_make)
remaining_makes = {}
for model in remaining_models_without_matched_make:
    for make in accident_matched_file["Manufacturer"]:
        make_split = make.split()
        model_split = model.split()
        x = True
        if len(make_split) < len(model_split):
            for i in range(len(make_split)):
                x &= make_split[i] == model_split[i]
            if x:
                if make in remaining_makes.keys():
                    remaining_makes[make].append(model)
                else:
                    remaining_makes[make] = [model]
print(len(remaining_makes))
print(len(remaining_models_without_matched_make))
for m in remaining_makes.keys():
    remaining_makes[m] = list(set(remaining_makes[m]))
mean, std = [], []
for factor in distance_factors:
    mean.append(accident_matched_file[factor].mean())
    std.append(accident_matched_file[factor].std())
for make in remaining_makes.keys():
    print("MAKE PHASE")
    print(make)
    closest_make_dist, closest_make = 1000, ""
    make_df = accident_matched_file[accident_matched_file["Manufacturer"]==make][distance_factors+["GenModel","Accident_rating"]]
    for i in range(len(distance_factors)):
        make_df[distance_factors[i]] = (make_df[distance_factors[i]]-mean[i])/std[i]
    make_mean_point = []
    for f in distance_factors:
        make_mean_point.append(make_df[f].mean())
    print(make_mean_point)
    # print("Makes:")
    for other_make in matched_manufacturers:
        # print(other_make)
        other_make_df = accident_matched_file[accident_matched_file["Manufacturer"]==other_make][distance_factors+["GenModel"]]
        for i in range(len(distance_factors)):
            other_make_df[distance_factors[i]] = (other_make_df[distance_factors[i]]-mean[i])/std[i]
        other_make_mean_point = []
        for f in distance_factors:
            other_make_mean_point.append(other_make_df[f].mean())
        # print(other_make_mean_point)
        dist = 0
        for i in range(len(make_mean_point)):
            if not math.isnan(make_mean_point[i]):
                dist += math.pow((make_mean_point[i]-other_make_mean_point[i]),2)
        dist = math.sqrt(dist)
        # print(dist)
        if closest_make_dist > dist and dist != 0:
            closest_make_dist = dist
            closest_make = other_make
    print("Closest make:")
    print(closest_make)
    closest_make_df = accident_matched_file[accident_matched_file["Manufacturer"]==closest_make][distance_factors+["GenModel","Accident_rating"]]
    print("MODEL PHASE:")
    for model in remaining_makes[make]:
        closest_dist, closest_point, closest_model = 1000, [], ""
        print(model)
        new_model_df = accident_matched_file[accident_matched_file["GenModel"]==model][distance_factors]
        for i in range(len(distance_factors)):
            new_model_df[distance_factors[i]] = (new_model_df[distance_factors[i]]-mean[i])/std[i]
        model_mean_point = []
        for f in distance_factors:
            model_mean_point.append(new_model_df[f].mean())
        print(model_mean_point)
        matched_model_df = closest_make_df[closest_make_df["Accident_rating"].notna()]
        matched_model_list = list(set(matched_model_df["GenModel"]))
        for matched_model in matched_model_list:
            current_matched_model_df = matched_model_df[matched_model_df["GenModel"]==matched_model]
            for i in range(len(distance_factors)):
                current_matched_model_df[distance_factors[i]] = (current_matched_model_df[distance_factors[i]]-mean[i])/std[i]
            matched_mean_point = []
            for f in distance_factors:
                matched_mean_point.append(current_matched_model_df[f].mean())
            dist = 0
            for i in range(len(model_mean_point)):
                if not math.isnan(model_mean_point[i]):
                    dist += math.pow((model_mean_point[i]-matched_mean_point[i]),2)
            dist = math.sqrt(dist)
            if closest_dist > dist and dist != 0:
                closest_dist = dist
                closest_point = matched_mean_point
                closest_model = matched_model
            mx, commonality_match = 0.875, closest_model
        for j, row in accident_values_df.iterrows():
            veh = row["GenModel"]
            val = SequenceMatcher(None, veh, closest_model).ratio()
            if veh in closest_model:
                val = 1
            if val > mx:
                mx = val
                commonality_match = veh
        print(closest_model)
        # print(closest_point)
        # print(closest_dist)
        commonality_detriment = []
        for y in years:
            if commonality_match in commonality[y].keys():
                commonality_detriment.append(commonality[y][commonality_match])
        commonality_detriment = sum(commonality_detriment)/(len(commonality_detriment)*2)
        matched_ratings[model] = int(list(accident_matched_file[accident_matched_file["GenModel"]==closest_model]["Accident_rating"])[0]-commonality_detriment)
        print(matched_ratings[model])
    print()

In [ ]:
for k in matched_ratings.keys():
    accident_matched_file.loc[accident_matched_file['GenModel'] == k, 'Accident_rating'] = matched_ratings[k]

In [ ]:
matched_models = list(set(accident_matched_file["GenModel"]))
continuous_factors = ["Engine_Power_BHP", "Weight_Kerb", "Volume_Box", "Top_Speed"]
model_factor_means = {}
for m in matched_models:
    matched_model_df = accident_matched_file[accident_matched_file["GenModel"] == m]
    model_factor_means[m] = {"Engine_Power_BHP": [matched_model_df["Engine_Power_BHP"].mean(), matched_model_df["Engine_Power_BHP"].std()],
                             "Weight_Kerb": [matched_model_df["Weight_Kerb"].mean(), matched_model_df["Weight_Kerb"].std()],
                             "Volume_Box": [matched_model_df["Volume_Box"].mean(), matched_model_df["Volume_Box"].std()],
                             "Top_Speed": [matched_model_df["Top_Speed"].mean(), matched_model_df["Top_Speed"].std()],
                             "Acceleration": [matched_model_df["Acceleration"].mean(), matched_model_df["Acceleration"].std()]}

In [ ]:
# value correction
accident_values = {"TVI_Code": [], "Accident_rating": []}
for i, row in accident_matched_file.iterrows():
    rating = row["Accident_rating"]
    if not math.isnan(rating) and rating < 99:
        rating += -2 if row["Transmission_Group"]=="Manual" else 2 if row["Transmission_Group"]=="Automatic" else 0
        for f in continuous_factors:
            if not math.isnan(row[f]) and not math.isnan(model_factor_means[row["GenModel"]][f][0]) and not math.isnan(model_factor_means[row["GenModel"]][f][1]) and model_factor_means[row["GenModel"]][f][1] != 0:
                rating += int((row[f]-model_factor_means[row["GenModel"]][f][0])/model_factor_means[row["GenModel"]][f][1])
        if not math.isnan(model_factor_means[row["GenModel"]]["Acceleration"][1]) and model_factor_means[row["GenModel"]]["Acceleration"][1] != 0:
            rating += int(abs((row["Acceleration"]-model_factor_means[row["GenModel"]]["Acceleration"][0])/model_factor_means[row["GenModel"]]["Acceleration"][1]))
    if not math.isnan(rating):
        accident_values["TVI_Code"].append(row["TVI_Code"])
        accident_values["Accident_rating"].append(rating)
accident_matched_file = accident_matched_file.drop(columns={"Accident_rating"})
accident_matched_file = accident_matched_file.merge(pd.DataFrame.from_dict(accident_values), on="TVI_Code", how="left")

In [ ]:
len(accident_matched_file["Accident_rating"].dropna())==len(data_file)

In [ ]:
pd.DataFrame.from_dict(accident_values).to_csv(r"C:\Users\JoshM\Documents\Vehicle File\Rating factors\factors\Accident_ratings.csv")

#### Injury calculation

In [ ]:
# direct match
casualty_matched_file = data_file.merge(casualty_values_df, on='GenModel', how="left")

In [ ]:
set(casualty_matched_file["Casualty_rating"].dropna())

In [ ]:
# bespoke value fill
luxury_makes = ["FERRARI", "BUGATTI", "LAMBORGHINI", "ASTON MARTIN", "T.V.R.", "ROLLS", "MCLAREN", "KOENIGSEGG", "DE TOMASO"]
for make in luxury_makes:
    casualty_matched_file.loc[casualty_matched_file['Manufacturer'] == make, "Casualty_rating"] = 99

In [ ]:
# indirect match

indirect_matches = {}
remaining_df = casualty_matched_file[casualty_matched_file["Casualty_rating"].isna()]
remaining_models = list(set(remaining_df["GenModel"]))
print(len(remaining_models))
for i in range(len(remaining_models)):
    if i % 100 == 0:
        print(i)
    mx, match = 0.875, 0
    for j, row in casualty_values_df.iterrows():
        veh = row["GenModel"]
        val = SequenceMatcher(None, veh, remaining_models[i]).ratio()
        if veh in remaining_models[i]:
            val = 1
        if val > mx:
            mx = val
            match = veh
    if match != 0:
        # print(remaining_models[i] + " - " + match)
        rarity_detriment = []
        indirect_matches[remaining_models[i]] = list(casualty_values_df[casualty_values_df["GenModel"]==match]["Casualty_rating"])[0]
for k in indirect_matches.keys():
    casualty_matched_file.loc[casualty_matched_file['GenModel'] == k, "Casualty_rating"] = indirect_matches[k]

In [ ]:
# ## Manufacturer vehicle imputation
remaining_models = list(set(casualty_matched_file[casualty_matched_file["Casualty_rating"].isna()]["GenModel"]))
matched_manufacturers = list(set(casualty_matched_file[casualty_matched_file["Casualty_rating"].notna()][casualty_matched_file["Casualty_rating"] < 99]["Manufacturer"]))
remaining_models_with_matched_make = []
for model in remaining_models:
    x = False
    for make in matched_manufacturers:
        x |= model.split()[0] in make.split()
    if x:
        remaining_models_with_matched_make.append(model)
print(len(remaining_models_with_matched_make))
distance_factors = ["Volume_Box","Weight_Kerb","Acceleration","Top_Speed"]
matched_ratings = {}
for model in remaining_models_with_matched_make:
    print(model)
    closest_dist, closest_point, closest_model = 100, [], ""
    model_make = ""
    for make in matched_manufacturers:
        make_split = make.split()
        model_split = model.split()
        x = True
        if len(make_split) < len(model_split):
            for i in range(len(make_split)):
                x &= make_split[i] == model_split[i]
            if x:
                model_make = make
    print(model_make)
    make_df = casualty_matched_file[casualty_matched_file["Manufacturer"]==model_make][distance_factors+["GenModel","Casualty_rating"]]
    mean, std = [], []
    for factor in distance_factors:
        mean.append(make_df[factor].mean())
        std.append(make_df[factor].std())
    new_model_df = casualty_matched_file[casualty_matched_file["GenModel"]==model][distance_factors]
    for i in range(len(distance_factors)):
        new_model_df[distance_factors[i]] = (new_model_df[distance_factors[i]]-mean[i])/std[i]
    mean_point = []
    for f in distance_factors:
        mean_point.append(new_model_df[f].mean())
    # print(mean_point)
    matched_model_df = make_df[make_df["Casualty_rating"].notna()]
    matched_model_list = list(set(matched_model_df["GenModel"]))
    # print("MATCHED MODEL")
    for matched_model in matched_model_list:
        current_matched_model_df = matched_model_df[matched_model_df["GenModel"]==matched_model]
        for i in range(len(distance_factors)):
            current_matched_model_df[distance_factors[i]] = (current_matched_model_df[distance_factors[i]]-mean[i])/std[i]
        matched_mean_point = []
        for f in distance_factors:
            matched_mean_point.append(current_matched_model_df[f].mean())
        dist = 0
        for i in range(len(mean_point)):
            if not math.isnan(mean_point[i]):
                dist += math.pow((mean_point[i]-matched_mean_point[i]),2)
        dist = math.sqrt(dist)
        if closest_dist > dist:
            closest_dist = dist
            closest_point = matched_mean_point
            closest_model = matched_model
    mx, commonality_match = 0.875, closest_model
    for j, row in casualty_values_df.iterrows():
        veh = row["GenModel"]
        val = SequenceMatcher(None, veh, closest_model).ratio()
        if veh in closest_model:
            val = 1
        if val > mx:
            mx = val
            commonality_match = veh
    print(closest_model)
    print(commonality_match)
    # print(closest_point)
    # print(closest_dist)
    commonality_detriment = []
    for y in years:
        if commonality_match in commonality[y].keys():
            commonality_detriment.append(commonality[y][commonality_match])
    commonality_detriment = sum(commonality_detriment)/(len(commonality_detriment)*2)
    matched_ratings[model] = int(list(casualty_matched_file[casualty_matched_file["GenModel"]==closest_model]["Casualty_rating"])[0]-commonality_detriment)
    print(matched_ratings[model])
    print()
print(matched_ratings)

In [ ]:
# Non-manufacturer vehicle imputation
remaining_models_without_matched_make = set(remaining_models)-set(remaining_models_with_matched_make)
remaining_makes = {}
for model in remaining_models_without_matched_make:
    for make in casualty_matched_file["Manufacturer"]:
        make_split = make.split()
        model_split = model.split()
        x = True
        if len(make_split) < len(model_split):
            for i in range(len(make_split)):
                x &= make_split[i] == model_split[i]
            if x:
                if make in remaining_makes.keys():
                    remaining_makes[make].append(model)
                else:
                    remaining_makes[make] = [model]
print(len(remaining_makes))
print(len(remaining_models_without_matched_make))
for m in remaining_makes.keys():
    remaining_makes[m] = list(set(remaining_makes[m]))
mean, std = [], []
for factor in distance_factors:
    mean.append(casualty_matched_file[factor].mean())
    std.append(casualty_matched_file[factor].std())
for make in remaining_makes.keys():
    print("MAKE PHASE")
    print(make)
    closest_make_dist, closest_make = 1000, ""
    make_df = casualty_matched_file[casualty_matched_file["Manufacturer"]==make][distance_factors+["GenModel","Casualty_rating"]]
    for i in range(len(distance_factors)):
        make_df[distance_factors[i]] = (make_df[distance_factors[i]]-mean[i])/std[i]
    make_mean_point = []
    for f in distance_factors:
        make_mean_point.append(make_df[f].mean())
    print(make_mean_point)
    # print("Makes:")
    for other_make in matched_manufacturers:
        # print(other_make)
        other_make_df = casualty_matched_file[casualty_matched_file["Manufacturer"]==other_make][distance_factors+["GenModel"]]
        for i in range(len(distance_factors)):
            other_make_df[distance_factors[i]] = (other_make_df[distance_factors[i]]-mean[i])/std[i]
        other_make_mean_point = []
        for f in distance_factors:
            other_make_mean_point.append(other_make_df[f].mean())
        # print(other_make_mean_point)
        dist = 0
        for i in range(len(make_mean_point)):
            if not math.isnan(make_mean_point[i]):
                dist += math.pow((make_mean_point[i]-other_make_mean_point[i]),2)
        dist = math.sqrt(dist)
        # print(dist)
        if closest_make_dist > dist and dist != 0:
            closest_make_dist = dist
            closest_make = other_make
    print("Closest make:")
    print(closest_make)
    closest_make_df = casualty_matched_file[casualty_matched_file["Manufacturer"]==closest_make][distance_factors+["GenModel","Casualty_rating"]]
    print("MODEL PHASE:")
    for model in remaining_makes[make]:
        closest_dist, closest_point, closest_model = 1000, [], ""
        print(model)
        new_model_df = casualty_matched_file[casualty_matched_file["GenModel"]==model][distance_factors]
        for i in range(len(distance_factors)):
            new_model_df[distance_factors[i]] = (new_model_df[distance_factors[i]]-mean[i])/std[i]
        model_mean_point = []
        for f in distance_factors:
            model_mean_point.append(new_model_df[f].mean())
        print(model_mean_point)
        matched_model_df = closest_make_df[closest_make_df["Casualty_rating"].notna()]
        matched_model_list = list(set(matched_model_df["GenModel"]))
        for matched_model in matched_model_list:
            current_matched_model_df = matched_model_df[matched_model_df["GenModel"]==matched_model]
            for i in range(len(distance_factors)):
                current_matched_model_df[distance_factors[i]] = (current_matched_model_df[distance_factors[i]]-mean[i])/std[i]
            matched_mean_point = []
            for f in distance_factors:
                matched_mean_point.append(current_matched_model_df[f].mean())
            dist = 0
            for i in range(len(model_mean_point)):
                if not math.isnan(model_mean_point[i]):
                    dist += math.pow((model_mean_point[i]-matched_mean_point[i]),2)
            dist = math.sqrt(dist)
            if closest_dist > dist and dist != 0:
                closest_dist = dist
                closest_point = matched_mean_point
                closest_model = matched_model
            mx, commonality_match = 0.875, closest_model
        for j, row in casualty_values_df.iterrows():
            veh = row["GenModel"]
            val = SequenceMatcher(None, veh, closest_model).ratio()
            if veh in closest_model:
                val = 1
            if val > mx:
                mx = val
                commonality_match = veh
        print(closest_model)
        # print(closest_point)
        # print(closest_dist)
        commonality_detriment = []
        for y in years:
            if commonality_match in commonality[y].keys():
                commonality_detriment.append(commonality[y][commonality_match])
        commonality_detriment = sum(commonality_detriment)/(len(commonality_detriment)*2)
        matched_ratings[model] = int(list(casualty_matched_file[casualty_matched_file["GenModel"]==closest_model]["Casualty_rating"])[0]-commonality_detriment)
        print(matched_ratings[model])
    print()

In [ ]:
for k in matched_ratings.keys():
    casualty_matched_file.loc[casualty_matched_file['GenModel'] == k, 'Casualty_rating'] = matched_ratings[k]

In [ ]:
matched_models = list(set(casualty_matched_file["GenModel"]))
continuous_factors = ["Engine_Power_BHP", "Weight_Kerb", "Volume_Box", "Top_Speed"]
model_factor_means = {}
for m in matched_models:
    matched_model_df = casualty_matched_file[casualty_matched_file["GenModel"] == m]
    model_factor_means[m] = {"Engine_Power_BHP": [matched_model_df["Engine_Power_BHP"].mean(), matched_model_df["Engine_Power_BHP"].std()],
                             "Weight_Kerb": [matched_model_df["Weight_Kerb"].mean(), matched_model_df["Weight_Kerb"].std()],
                             "Volume_Box": [matched_model_df["Volume_Box"].mean(), matched_model_df["Volume_Box"].std()],
                             "Top_Speed": [matched_model_df["Top_Speed"].mean(), matched_model_df["Top_Speed"].std()],
                             "Acceleration": [matched_model_df["Acceleration"].mean(), matched_model_df["Acceleration"].std()]}

In [ ]:
# value correction
casualty_values = {"TVI_Code": [], "Injury_rating": []}
for i, row in casualty_matched_file.iterrows():
    rating = row["Casualty_rating"]
    if not math.isnan(rating) and rating < 99:
        rating += -2 if row["Transmission_Group"]=="Manual" else 2 if row["Transmission_Group"]=="Automatic" else 0
        for f in continuous_factors:
            if not math.isnan(row[f]) and not math.isnan(model_factor_means[row["GenModel"]][f][0]) and not math.isnan(model_factor_means[row["GenModel"]][f][1]) and model_factor_means[row["GenModel"]][f][1] != 0:
                rating += int((row[f]-model_factor_means[row["GenModel"]][f][0])/model_factor_means[row["GenModel"]][f][1])
        if not math.isnan(model_factor_means[row["GenModel"]]["Acceleration"][1]) and model_factor_means[row["GenModel"]]["Acceleration"][1] != 0:
            rating += int(abs((row["Acceleration"]-model_factor_means[row["GenModel"]]["Acceleration"][0])/model_factor_means[row["GenModel"]]["Acceleration"][1]))
    if not math.isnan(rating):
        casualty_values["TVI_Code"].append(row["TVI_Code"])
        casualty_values["Injury_rating"].append(rating)
casualty_matched_file = casualty_matched_file.drop(columns={"Casualty_rating"})
casualty_matched_file = casualty_matched_file.merge(pd.DataFrame.from_dict(casualty_values), on="TVI_Code", how="left")

In [ ]:
len(casualty_matched_file["Injury_rating"].dropna())==len(data_file)

In [ ]:
pd.DataFrame.from_dict(casualty_values).to_csv(r"C:\Users\JoshM\Documents\Vehicle File\Rating factors\factors\Casualty_ratings.csv")

#### Theft calculation

In [ ]:
# direct match
theft_matched_file = data_file.merge(theft_values_df, on='GenModel', how="left")

In [ ]:
set(theft_matched_file["Theft_rating"].dropna())

In [ ]:
# bespoke value fill
luxury_makes = ["FERRARI", "BUGATTI", "LAMBORGHINI", "ASTON MARTIN", "T.V.R.", "ROLLS", "MCLAREN", "KOENIGSEGG", "DE TOMASO"]
for make in luxury_makes:
    theft_matched_file.loc[theft_matched_file['Manufacturer'] == make, "Theft_rating"] = 99

#### Provisional theft calculation

In [ ]:
# direct match
theft_matched_file = data_file.merge(theft_values_df, on='GenModel', how="left")

In [ ]:
set(theft_matched_file["Theft_rating"].dropna())

In [ ]:
# indirect match

indirect_matches = {}
remaining_df = theft_matched_file[theft_matched_file["Theft_rating"].isna()]
remaining_models = list(set(remaining_df["GenModel"]))
print(len(remaining_models))
for i in range(len(remaining_models)):
    if i % 100 == 0:
        print(i)
    mx, match = 0.875, 0
    for j, row in theft_values_df.iterrows():
        veh = row["GenModel"]
        val = SequenceMatcher(None, veh, remaining_models[i]).ratio()
        if veh in remaining_models[i]:
            val = 1
        if val > mx:
            mx = val
            match = veh
    if match != 0:
        print(remaining_models[i] + " - " + match)
        rarity_detriment = []
        indirect_matches[remaining_models[i]] = list(theft_values_df[theft_values_df["GenModel"]==match]["Theft_rating"])[0]
for k in indirect_matches.keys():
    theft_matched_file.loc[theft_matched_file['GenModel'] == k, "Theft_rating"] = indirect_matches[k]

In [ ]:
# bespoke value fill
luxury_makes = ["FERRARI", "BUGATTI", "LAMBORGHINI", "ASTON MARTIN", "T.V.R.", "ROLLS", "MCLAREN", "KOENIGSEGG", "DE TOMASO"]
for make in luxury_makes:
    theft_matched_file.loc[theft_matched_file['Manufacturer'] == make, "Theft_rating"] = 99

In [ ]:
# ## Manufacturer vehicle imputation
remaining_models = list(set(theft_matched_file[theft_matched_file["Theft_rating"].isna()]["GenModel"]))
matched_manufacturers = list(set(theft_matched_file[theft_matched_file["Theft_rating"].notna()][theft_matched_file["Theft_rating"] < 99]["Manufacturer"]))
remaining_models_with_matched_make = []
for model in remaining_models:
    x = False
    for make in matched_manufacturers:
        x |= model.split()[0] in make.split()
    if x:
        remaining_models_with_matched_make.append(model)
print(len(remaining_models_with_matched_make))
distance_factors = ["Volume_Box","Weight_Kerb","Acceleration","Top_Speed"]
matched_ratings = {}
for model in remaining_models_with_matched_make:
    print(model)
    closest_dist, closest_point, closest_model = 100, [], ""
    model_make = ""
    for make in matched_manufacturers:
        make_split = make.split()
        model_split = model.split()
        x = True
        if len(make_split) < len(model_split):
            for i in range(len(make_split)):
                x &= make_split[i] == model_split[i]
            if x:
                model_make = make
    print(model_make)
    make_df = theft_matched_file[theft_matched_file["Manufacturer"]==model_make][distance_factors+["GenModel","Theft_rating"]]
    mean, std = [], []
    for factor in distance_factors:
        mean.append(make_df[factor].mean())
        std.append(make_df[factor].std())
    new_model_df = theft_matched_file[theft_matched_file["GenModel"]==model][distance_factors]
    for i in range(len(distance_factors)):
        new_model_df[distance_factors[i]] = (new_model_df[distance_factors[i]]-mean[i])/std[i]
    mean_point = []
    for f in distance_factors:
        mean_point.append(new_model_df[f].mean())
    # print(mean_point)
    matched_model_df = make_df[make_df["Theft_rating"].notna()]
    matched_model_list = list(set(matched_model_df["GenModel"]))
    # print("MATCHED MODEL")
    for matched_model in matched_model_list:
        current_matched_model_df = matched_model_df[matched_model_df["GenModel"]==matched_model]
        for i in range(len(distance_factors)):
            current_matched_model_df[distance_factors[i]] = (current_matched_model_df[distance_factors[i]]-mean[i])/std[i]
        matched_mean_point = []
        for f in distance_factors:
            matched_mean_point.append(current_matched_model_df[f].mean())
        dist = 0
        for i in range(len(mean_point)):
            if not math.isnan(mean_point[i]):
                dist += math.pow((mean_point[i]-matched_mean_point[i]),2)
        dist = math.sqrt(dist)
        if closest_dist > dist:
            closest_dist = dist
            closest_point = matched_mean_point
            closest_model = matched_model
    mx, commonality_match = 0.875, closest_model
    for j, row in theft_values_df.iterrows():
        veh = row["GenModel"]
        val = SequenceMatcher(None, veh, closest_model).ratio()
        if veh in closest_model:
            val = 1
        if val > mx:
            mx = val
            commonality_match = veh
    print(closest_model)
    print(commonality_match)
    # print(closest_point)
    # print(closest_dist)
    matched_ratings[model] = int(list(theft_matched_file[theft_matched_file["GenModel"]==closest_model]["Theft_rating"])[0]-(commonality[commonality_match]/2))
    print(matched_ratings[model])
    print()
print(matched_ratings)

In [ ]:
# Non-manufacturer vehicle imputation
remaining_models_without_matched_make = set(remaining_models)-set(remaining_models_with_matched_make)
remaining_makes = {}
for model in remaining_models_without_matched_make:
    for make in theft_matched_file["Manufacturer"]:
        make_split = make.split()
        model_split = model.split()
        x = True
        if len(make_split) < len(model_split):
            for i in range(len(make_split)):
                x &= make_split[i] == model_split[i]
            if x:
                if make in remaining_makes.keys():
                    remaining_makes[make].append(model)
                else:
                    remaining_makes[make] = [model]
print(len(remaining_makes))
print(len(remaining_models_without_matched_make))
for m in remaining_makes.keys():
    remaining_makes[m] = list(set(remaining_makes[m]))
mean, std = [], []
for factor in distance_factors:
    mean.append(theft_matched_file[factor].mean())
    std.append(theft_matched_file[factor].std())
for make in remaining_makes.keys():
    print("MAKE PHASE")
    print(make)
    closest_make_dist, closest_make = 1000, ""
    make_df = theft_matched_file[theft_matched_file["Manufacturer"]==make][distance_factors+["GenModel","Theft_rating"]]
    for i in range(len(distance_factors)):
        make_df[distance_factors[i]] = (make_df[distance_factors[i]]-mean[i])/std[i]
    make_mean_point = []
    for f in distance_factors:
        make_mean_point.append(make_df[f].mean())
    print(make_mean_point)
    # print("Makes:")
    for other_make in matched_manufacturers:
        # print(other_make)
        other_make_df = theft_matched_file[theft_matched_file["Manufacturer"]==other_make][distance_factors+["GenModel"]]
        for i in range(len(distance_factors)):
            other_make_df[distance_factors[i]] = (other_make_df[distance_factors[i]]-mean[i])/std[i]
        other_make_mean_point = []
        for f in distance_factors:
            other_make_mean_point.append(other_make_df[f].mean())
        # print(other_make_mean_point)
        dist = 0
        for i in range(len(make_mean_point)):
            if not math.isnan(make_mean_point[i]):
                dist += math.pow((make_mean_point[i]-other_make_mean_point[i]),2)
        dist = math.sqrt(dist)
        # print(dist)
        if closest_make_dist > dist and dist != 0:
            closest_make_dist = dist
            closest_make = other_make
    print("Closest make:")
    print(closest_make)
    closest_make_df = theft_matched_file[theft_matched_file["Manufacturer"]==closest_make][distance_factors+["GenModel","Theft_rating"]]
    print("MODEL PHASE:")
    for model in remaining_makes[make]:
        closest_dist, closest_point, closest_model = 1000, [], ""
        print(model)
        new_model_df = theft_matched_file[theft_matched_file["GenModel"]==model][distance_factors]
        for i in range(len(distance_factors)):
            new_model_df[distance_factors[i]] = (new_model_df[distance_factors[i]]-mean[i])/std[i]
        model_mean_point = []
        for f in distance_factors:
            model_mean_point.append(new_model_df[f].mean())
        print(model_mean_point)
        matched_model_df = closest_make_df[closest_make_df["Theft_rating"].notna()]
        matched_model_list = list(set(matched_model_df["GenModel"]))
        for matched_model in matched_model_list:
            current_matched_model_df = matched_model_df[matched_model_df["GenModel"]==matched_model]
            for i in range(len(distance_factors)):
                current_matched_model_df[distance_factors[i]] = (current_matched_model_df[distance_factors[i]]-mean[i])/std[i]
            matched_mean_point = []
            for f in distance_factors:
                matched_mean_point.append(current_matched_model_df[f].mean())
            dist = 0
            for i in range(len(model_mean_point)):
                if not math.isnan(model_mean_point[i]):
                    dist += math.pow((model_mean_point[i]-matched_mean_point[i]),2)
            dist = math.sqrt(dist)
            if closest_dist > dist and dist != 0:
                closest_dist = dist
                closest_point = matched_mean_point
                closest_model = matched_model
            mx, commonality_match = 0.875, closest_model
        for j, row in theft_values_df.iterrows():
            veh = row["GenModel"]
            val = SequenceMatcher(None, veh, closest_model).ratio()
            if veh in closest_model:
                val = 1
            if val > mx:
                mx = val
                commonality_match = veh
        print(closest_model)
        # print(closest_point)
        # print(closest_dist)
        matched_ratings[model] = int(list(theft_matched_file[theft_matched_file["GenModel"]==closest_model]["Theft_rating"])[0]-commonality[commonality_match])
        print(matched_ratings[model])
    print()

In [ ]:
for k in matched_ratings.keys():
    theft_matched_file.loc[theft_matched_file['GenModel'] == k, 'Theft_rating'] = matched_ratings[k]

In [ ]:
for i, row in theft_matched_file.iterrows():
    if i % 1000 == 0:
        print(i)
    if not math.isnan(row["Theft_rating"]) and row["Manufacturer"] not in luxury_makes:
        band = manufacturer_band[row["Manufacturer"]]
        year = int(row["Start_Year_Veh_Generation"])
        shift = 0
        abs_dif = abs(year-date_shift[band][0])
        if year < date_shift[band][0]:
            shift = date_shift[band][1]+(date_shift[band][2][0]*abs_dif)
        else:
            shift = date_shift[band][1]+(date_shift[band][2][1]*abs_dif)
        shift = shift if shift <= 30 else 30
        shift = shift if shift > 0 else 0
        theft_matched_file.loc[i, "Theft_rating"] = row["Theft_rating"]+shift

In [ ]:
for i, row in theft_matched_file.iterrows():
    if i % 1000 == 0:
        print(i)
    theft_matched_file.loc[i, "Theft_rating"] = int(row["Theft_rating"]) if row["Theft_rating"]>0 else 1

In [ ]:
len(theft_matched_file["Theft_rating"])==len(theft_matched_file["Theft_rating"].dropna())

In [ ]:
theft_values = {"TVI_Code": [], "Theft_rating": []}
for i, row in theft_matched_file.iterrows():
    theft_values["TVI_Code"].append(row["TVI_Code"])
    theft_values["Theft_rating"].append(row["Theft_rating"])

In [ ]:
pd.DataFrame.from_dict(theft_values).to_csv(r"C:\Users\JoshM\Documents\Vehicle File\Rating factors\factors\Theft_ratings.csv")

### Data file output

In [ ]:
output = data_file.merge(pd.DataFrame.from_dict(casualty_values), on="TVI_Code", how="left")
output = output.merge(pd.DataFrame.from_dict(accident_values), on="TVI_Code", how="left")
output = output.merge(pd.DataFrame.from_dict(theft_values), on="TVI_Code", how="left")
output.to_csv(r"C:\Users\JoshM\Documents\Vehicle File\Rating factors\data\provisional_ratings.csv")